<a href="https://colab.research.google.com/github/AntMilton/MoonIa/blob/main/Moonv1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# 1. SETUP - IMPORTAÇÃO DAS BIBLIOTECAS NECESSÁRIAS
# ==============================================================================
import pandas as pd
import numpy as np

# Bibliotecas de Machine Learning e Pré-processamento
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest

# Bibliotecas para o Autoencoder (Deep Learning)
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

print("Bibliotecas importadas com sucesso.")

# ==============================================================================
# 2. CARREGAMENTO DOS DADOS (SIMULAÇÃO)
# !! SUBSTITUA ESTE BLOCO PELO SEU CÓDIGO DE CARREGAMENTO DE DADOS !!
# ==============================================================================
print("\n--- A simular dados de exemplo... ---")
num_registos = 10000
data = {
    'id_transacao': range(num_registos),
    'data_hora': pd.to_datetime(np.random.choice(pd.date_range('2024-01-01', '2025-01-01', freq='h'), num_registos)),
    'valor': np.random.lognormal(mean=8, sigma=1.5, size=num_registos).round(2),
    'tipo_transacao': np.random.choice(['Pagamento', 'Recebimento', 'Transferência'], size=num_registos),
    'categoria': np.random.choice(['Fornecedores', 'Salários', 'Marketing', 'Diversos', 'Outros'], size=num_registos, p=[0.4, 0.2, 0.2, 0.1, 0.1]),
    'id_entidade': np.random.randint(1, 50, size=num_registos)
}
df = pd.DataFrame(data)

# Inserir alguns valores anómalos para teste
df.loc[df.sample(frac=0.02).index, 'valor'] *= np.random.uniform(5, 10) # Valores muito altos
df.loc[df.sample(frac=0.01).index, 'valor'] *= -1 # Valores negativos
df.loc[df.sample(frac=0.01).index, 'data_hora'] += pd.to_timedelta(np.random.randint(21, 23), unit='h') # Horas incomuns
print(f"Dados simulados criados com {len(df)} registos.")
print("!! Lembre-se de substituir este bloco pelos seus dados reais. !!")


# ==============================================================================
# 3. ENGENHARIA DE ATRIBUTOS (FEATURE ENGINEERING)
# ==============================================================================
print("\n--- A iniciar a Engenharia de Atributos... ---")

# 3.1. Garantir que a coluna de data está no formato datetime
df['data_hora'] = pd.to_datetime(df['data_hora'])

# 3.2. Refinamento de Features Existentes e Criação de Novas
# Desvio em relação à média (como no seu TCC)
media_por_entidade = df.groupby('id_entidade')['valor'].transform('mean')
df['desvio_media_historica'] = (df['valor'] - media_por_entidade) / (media_por_entidade + 1e-6) # Adicionado 1e-6 para evitar divisão por zero

# Desvio em relação à mediana (nova feature)
mediana_por_entidade = df.groupby('id_entidade')['valor'].transform('median')
df['desvio_mediana_historica'] = (df['valor'] - mediana_por_entidade) / (mediana_por_entidade + 1e-6)

# Features contextuais baseadas em regras de auditoria
df['hora_incomum'] = ((df['data_hora'].dt.hour < 7) | (df['data_hora'].dt.hour > 20)).astype(int)
df['dia_fim_de_semana'] = (df['data_hora'].dt.dayofweek >= 5).astype(int)
df['arredondamento_valor'] = (df['valor'] > 1000) & (df['valor'] % 1000 == 0).astype(int)

# Feature de sequência rápida (requer ordenação prévia)
df = df.sort_values(by=['id_entidade', 'data_hora'])
diferenca_tempo = df.groupby('id_entidade')['data_hora'].diff().dt.total_seconds()
df['sequencia_rapida'] = (diferenca_tempo < 60).astype(int)

# Limpeza de valores NaN que possam ter sido criados (ex: na primeira transação de uma entidade)
df.fillna(0, inplace=True)
print("Engenharia de Atributos concluída.")


# ==============================================================================
# 4. DEFINIÇÃO DA COMPLEXIDADE DA TRANSAÇÃO
# ==============================================================================
print("\n--- A definir a complexidade das transações... ---")

# Regra robusta com múltiplos critérios
media_geral = df['valor'].mean()
std_geral = df['valor'].std()

# Critério 1: Valor excede 3 desvios padrão da média geral
criterio_1 = df['valor'] > (media_geral + 3 * std_geral)

# Critério 2: Categoria é de alto risco ("Diversos" ou "Outros")
criterio_2 = df['categoria'].isin(['Diversos', 'Outros'])

# Critério 3: O valor da transação é negativo
criterio_3 = df['valor'] < 0

# Aplicar a regra: se qualquer critério for verdadeiro, a transação é complexa
df['complexidade'] = (criterio_1 | criterio_2 | criterio_3).astype(int) # 1 para complexa, 0 para simples

num_complexas = df['complexidade'].sum()
num_simples = len(df) - num_complexas
print(f"Transações classificadas: {num_complexas} complexas e {num_simples} simples.")


# ==============================================================================
# 5. PRÉ-PROCESSAMENTO E DIVISÃO DOS DADOS
# ==============================================================================
print("\n--- A pré-processar e a dividir os dados... ---")

# Selecionar as features numéricas para os modelos
features_numericas = df.select_dtypes(include=np.number).columns.tolist()
# Remover colunas que não são features, como IDs ou a própria label de complexidade
features_a_remover = ['id_transacao', 'id_entidade', 'complexidade']
features_para_modelo = [f for f in features_numericas if f not in features_a_remover]

X = df[features_para_modelo]

# Normalizar os dados
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

df_scaled = pd.DataFrame(X_scaled, columns=features_para_modelo, index=df.index)
df_scaled['complexidade'] = df['complexidade'] # Adicionar a coluna de complexidade ao df escalado

# Separar em dataframes para cada modelo
df_simples = df_scaled[df_scaled['complexidade'] == 0].drop('complexidade', axis=1)
df_complexas = df_scaled[df_scaled['complexidade'] == 1].drop('complexidade', axis=1)

# Divisão Treino/Teste para ambos os conjuntos
X_simples_train, X_simples_test = train_test_split(df_simples, test_size=0.3, random_state=42)
X_complexas_train, X_complexas_test = train_test_split(df_complexas, test_size=0.3, random_state=42)
print("Dados divididos em conjuntos de treino e teste para ambos os modelos.")


# ==============================================================================
# 6. MODELO 1: ISOLATION FOREST (TRANSAÇÕES SIMPLES)
# ==============================================================================
print("\n--- A treinar o modelo Isolation Forest... ---")

# Instanciar o modelo com os hiperparâmetros otimizados
iso_forest_otimizado = IsolationForest(n_estimators=200, contamination=0.01, random_state=42, n_jobs=-1)

# Treinar o modelo
iso_forest_otimizado.fit(X_simples_train)

# Fazer predições (-1 para anomalia, 1 para normal)
predicoes_if = iso_forest_otimizado.predict(X_simples_test)

# Guardar os resultados
resultados_if = X_simples_test.copy()
resultados_if['anomalia_detectada'] = predicoes_if
print("Modelo Isolation Forest treinado e predições realizadas.")


# ==============================================================================
# 7. MODELO 2: AUTOENCODER (TRANSAÇÕES COMPLEXAS)
# ==============================================================================
print("\n--- A construir e treinar o modelo Autoencoder... ---")

# 7.1. Definir a nova arquitetura de rede neural profunda
input_dim = X_complexas_train.shape[1]
input_layer = Input(shape=(input_dim,))
encoder = Dense(16, activation='relu')(input_layer)
encoder = Dense(8, activation='relu')(encoder) # Gargalo
decoder = Dense(16, activation='relu')(encoder)
decoder = Dense(input_dim, activation='linear')(decoder)
autoencoder_profundo = Model(inputs=input_layer, outputs=decoder)
autoencoder_profundo.compile(optimizer='adam', loss='mean_squared_error')
autoencoder_profundo.summary()

# 7.2. Treinar o modelo
autoencoder_profundo.fit(X_complexas_train, X_complexas_train,
                         epochs=50,
                         batch_size=32,
                         shuffle=True,
                         validation_data=(X_complexas_test, X_complexas_test),
                         verbose=1)

# 7.3. Fazer predições e calcular o erro de reconstrução
reconstrucoes = autoencoder_profundo.predict(X_complexas_test)
mse = np.mean(np.power(X_complexas_test - reconstrucoes, 2), axis=1)

# 7.4. Aplicar o novo limiar de 95%
threshold = np.quantile(mse, 0.95)
anomalias_ae = (mse > threshold).astype(int) # 1 para anomalia, 0 para normal

# Guardar os resultados
resultados_ae = X_complexas_test.copy()
resultados_ae['erro_reconstrucao'] = mse
resultados_ae['anomalia_detectada'] = anomalias_ae.replace({0: 1, 1: -1}) # Alinhar com o formato do IF (-1 para anomalia)
print("Modelo Autoencoder treinado e predições realizadas.")


# ==============================================================================
# 8. ANÁLISE FINAL DOS RESULTADOS
# ==============================================================================
print("\n--- Resultados Finais ---")

# Anomalias detetadas pelo Isolation Forest
anomalias_if_df = resultados_if[resultados_if['anomalia_detectada'] == -1]
print(f"\nO Isolation Forest detetou {len(anomalias_if_df)} anomalias em transações simples.")
# Mostrar as transações originais (não escaladas)
display(df.loc[anomalias_if_df.index].head())


# Anomalias detetadas pelo Autoencoder
anomalias_ae_df = resultados_ae[resultados_ae['anomalia_detectada'] == -1]
print(f"\nO Autoencoder detetou {len(anomalias_ae_df)} anomalias em transações complexas.")
# Mostrar as transações originais (não escaladas)
display(df.loc[anomalias_ae_df.index].head())

print("\nAnálise concluída. O próximo passo seria integrar estes resultados no dashboard.")

Bibliotecas importadas com sucesso.

--- A simular dados de exemplo... ---
Dados simulados criados com 10000 registos.
!! Lembre-se de substituir este bloco pelos seus dados reais. !!

--- A iniciar a Engenharia de Atributos... ---
Engenharia de Atributos concluída.

--- A definir a complexidade das transações... ---
Transações classificadas: 2149 complexas e 7851 simples.

--- A pré-processar e a dividir os dados... ---
Dados divididos em conjuntos de treino e teste para ambos os modelos.

--- A treinar o modelo Isolation Forest... ---
Modelo Isolation Forest treinado e predições realizadas.

--- A construir e treinar o modelo Autoencoder... ---


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 6)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 6)              │           102 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 494 (1.93 KB)

 Trainable params: 494 (1.93 KB)

...

Epoch 1/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step - loss: 0.1450 - val_loss: 0.0983
Epoch 2/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0881 - val_loss: 0.0565
Epoch 3/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 0.0521 - val_loss: 0.0315
Epoch 4/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.0241 - val_loss: 0.0068
Epoch 5/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0060 - val_loss: 0.0039
Epoch 6/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0036 - val_loss: 0.0031
Epoch 7/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0030 - val_loss: 0.0027
Epoch 8/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0020 - val_loss: 0.0026
Epoch 9/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0017 - val_loss: 0.0024
Epoch 10/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0018 - val_loss: 0.0023
Epoch 11/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0020 - val_loss: 0.0022
Epoch 12/50
47/47 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0014 - va

,id_transacao,data_hora,valor,tipo_transacao,categoria,id_entidade,desvio_media_historica,desvio_mediana_historica,hora_incomum,dia_fim_de_semana,arredondamento_valor,sequencia_rapida,complexidade
9381,9381,2024-10-13 23:00:00,71979.46,Transferência,Marketing,43,4.595297,22.854993,1,1,False,0,0
3672,3672,2024-02-04 23:00:00,73762.29,Transferência,Fornecedores,15,6.401311,24.097580,1,1,False,0,0
7437,7437,2024-03-21 18:00:00,81393.68,Transferência,Fornecedores,19,6.274835,21.182648,0,0,False,0,0
6680,6680,2024-10-13 02:00:00,76392.19,Recebimento,Fornecedores,13,5.636293,21.842300,1,1,False,0,0
6009,6009,2024-01-10 15:00:00,80697.25,Recebimento,Marketing,35,7.477505,26.726726,0,0,False,0,0



O Autoencoder detetou 33 anomalias em transações complexas.


,id_transacao,data_hora,valor,tipo_transacao,categoria,id_entidade,desvio_media_historica,desvio_mediana_historica,hora_incomum,dia_fim_de_semana,arredondamento_valor,sequencia_rapida,complexidade
4569,4569,2024-08-25 21:00:00,329861.72,Recebimento,Fornecedores,2,34.492966,127.449575,1,1,False,0,1
9090,9090,2024-12-05 12:00:00,683680.54,Transferência,Fornecedores,21,45.343893,249.965619,0,0,False,0,1
9575,9575,2024-10-13 01:00:00,142825.20,Transferência,Salários,20,12.997951,48.092887,1,1,False,0,1
6195,6195,2024-11-06 15:00:00,195090.44,Pagamento,Fornecedores,7,23.196194,63.520862,0,0,False,0,1
216,216,2024-09-22 05:00:00,166143.77,Recebimento,Fornecedores,7,19.606068,53.947537,1,1,False,0,1



Análise concluída. O próximo passo seria integrar estes resultados no dashboard.


In [ ]:
# ==============================================================================
# 1. SETUP - IMPORTAÇÃO DAS BIBLIOTECAS
# ==============================================================================
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import numpy as np

# Importações para o pipeline de Machine Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

print("Bibliotecas importadas com sucesso.")

# ==============================================================================
# 2. FUNÇÃO DO PIPELINE DE DADOS E MODELAGEM
# (Esta função encapsula todo o processo que definimos anteriormente)
# ==============================================================================

def preparar_dados_para_dashboard():
    """
    Executa o pipeline completo: simulação de dados, engenharia de atributos,
    modelagem e predição. Retorna um DataFrame final pronto para o dashboard.
    """
    print("\n--- A iniciar a preparação completa dos dados para o dashboard... ---")

    # --- Simulação de Dados (SUBSTITUA PELOS SEUS DADOS REAIS) ---
    print("--- A simular dados de exemplo... ---")
    num_registos = 10000
    data = {
        'id_transacao': range(num_registos),
        'data_hora': pd.to_datetime(np.random.choice(pd.date_range('2024-01-01', '2025-01-01', freq='h'), num_registos)),
        'valor': np.random.lognormal(mean=8, sigma=1.5, size=num_registos).round(2),
        'tipo_transacao': np.random.choice(['Pagamento', 'Recebimento', 'Transferência'], size=num_registos),
        'categoria': np.random.choice(['Fornecedores', 'Salários', 'Marketing', 'Diversos', 'Outros'], size=num_registos, p=[0.4, 0.2, 0.2, 0.1, 0.1]),
        'id_entidade': np.random.randint(1, 50, size=num_registos)
    }
    df = pd.DataFrame(data)
    df.loc[df.sample(frac=0.02).index, 'valor'] *= np.random.uniform(5, 10)
    df.loc[df.sample(frac=0.01).index, 'valor'] *= -1
    df.loc[df.sample(frac=0.01).index, 'data_hora'] += pd.to_timedelta(np.random.randint(21, 23), unit='h')
    print("!! Lembre-se de substituir este bloco pelos seus dados reais. !!")

    # --- Engenharia de Atributos ---
    print("--- Engenharia de Atributos... ---")
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    mediana_por_entidade = df.groupby('id_entidade')['valor'].transform('median')
    df['desvio_mediana_historica'] = (df['valor'] - mediana_por_entidade) / (mediana_por_entidade + 1e-6)
    df['hora_incomum'] = ((df['data_hora'].dt.hour < 7) | (df['data_hora'].dt.hour > 20)).astype(int)
    df['dia_fim_de_semana'] = (df['data_hora'].dt.dayofweek >= 5).astype(int)
    df['arredondamento_valor'] = (df['valor'] > 1000) & (df['valor'] % 1000 == 0).astype(int)
    df = df.sort_values(by=['id_entidade', 'data_hora'])
    diferenca_tempo = df.groupby('id_entidade')['data_hora'].diff().dt.total_seconds()
    df['sequencia_rapida'] = (diferenca_tempo < 60).astype(int)
    df.fillna(0, inplace=True)

    # --- Definição da Complexidade ---
    print("--- Definindo a Complexidade... ---")
    media_geral = df['valor'].mean()
    std_geral = df['valor'].std()
    criterio_1 = df['valor'] > (media_geral + 3 * std_geral)
    criterio_2 = df['categoria'].isin(['Diversos', 'Outros'])
    criterio_3 = df['valor'] < 0
    df['complexidade'] = np.where((criterio_1 | criterio_2 | criterio_3), 'Complexa', 'Simples')

    # --- Pré-processamento e Divisão ---
    features_para_modelo = df.select_dtypes(include=np.number).columns.drop(['id_transacao', 'id_entidade'])
    X = df[features_para_modelo]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    df_scaled = pd.DataFrame(X_scaled, columns=features_para_modelo, index=df.index)

    # Adicionar colunas originais necessárias para a lógica e o resultado final
    df_scaled[['complexidade', 'id_transacao']] = df[['complexidade', 'id_transacao']]

    # --- Treino e Predição ---
    print("--- Treinando e prevendo com os modelos... ---")
    # Isolation Forest
    df_simples = df_scaled[df_scaled['complexidade'] == 'Simples'].drop(['complexidade', 'id_transacao'], axis=1)
    iso_forest = IsolationForest(n_estimators=200, contamination=0.01, random_state=42, n_jobs=-1)
    iso_forest.fit(df_simples)
    predicoes_if = iso_forest.predict(df_simples)

    # Autoencoder
    df_complexas = df_scaled[df_scaled['complexidade'] == 'Complexa'].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_complexas.empty:
        input_dim = df_complexas.shape[1]
        input_layer = Input(shape=(input_dim,))
        encoder = Dense(16, activation='relu')(input_layer)
        encoder = Dense(8, activation='relu')(encoder)
        decoder = Dense(16, activation='relu')(encoder)
        decoder = Dense(input_dim, activation='linear')(decoder)
        autoencoder = Model(inputs=input_layer, outputs=decoder)
        autoencoder.compile(optimizer='adam', loss='mean_squared_error')
        autoencoder.fit(df_complexas, df_complexas, epochs=50, batch_size=32, verbose=0)
        reconstrucoes = autoencoder.predict(df_complexas)
        mse = np.mean(np.power(df_complexas - reconstrucoes, 2), axis=1)
        threshold = np.quantile(mse, 0.95)
        predicoes_ae = np.where(mse > threshold, -1, 1)
    else:
        predicoes_ae = np.array([])


    # --- Consolidação dos Resultados ---
    print("--- Consolidando resultados... ---")
    df['anomalia_detectada'] = 'Normal'
    df['modelo_deteccao'] = 'N/A'

    indices_simples = df_simples.index
    df.loc[indices_simples, 'anomalia_detectada'] = np.where(predicoes_if == -1, 'Anómala', 'Normal')
    df.loc[indices_simples, 'modelo_deteccao'] = 'Isolation Forest'

    if not df_complexas.empty:
        indices_complexas = df_complexas.index
        df.loc[indices_complexas, 'anomalia_detectada'] = np.where(predicoes_ae == -1, 'Anómala', 'Normal')
        df.loc[indices_complexas, 'modelo_deteccao'] = 'Autoencoder'

    print("Preparação dos dados concluída.")
    return df.sort_index()

# Executar a função para obter o DataFrame final
df_final = preparar_dados_para_dashboard()

# ==============================================================================
# 3. INICIALIZAÇÃO E LAYOUT DA APLICAÇÃO DASH
# ==============================================================================
app = dash.Dash(__name__)
app.title = "Dashboard de Auditoria Financeira"

app.layout = html.Div(style={'fontFamily': 'Arial, sans-serif', 'padding': '20px'}, children=[
    html.H1("Dashboard de Auditoria de Anomalias Financeiras", style={'textAlign': 'center', 'color': '#003366'}),
    html.H2("Caso de Estudo: System Business Development (SBD)", style={'textAlign': 'center', 'color': '#505050'}),

    html.Div(className='control-panel', style={'backgroundColor': '#f2f2f2', 'padding': '20px', 'borderRadius': '5px', 'marginBottom': '20px'}, children=[
        html.H4("Filtros de Análise", style={'marginTop': 0}),
        html.Div(className='filters', style={'display': 'flex', 'gap': '40px'}, children=[
            html.Div(children=[
                html.Label("Estado da Anomalia:"),
                dcc.Dropdown(
                    id='filtro-anomalia',
                    options=[
                        {'label': 'Todas', 'value': 'Todas'},
                        {'label': 'Anómalas', 'value': 'Anómala'},
                        {'label': 'Normais', 'value': 'Normal'}
                    ],
                    value='Todas'
                )
            ], style={'width': '33%'}),

            html.Div(children=[
                html.Label("Complexidade da Transação:"),
                dcc.Dropdown(
                    id='filtro-complexidade',
                    options=[
                        {'label': 'Todas', 'value': 'Todas'},
                        {'label': 'Simples', 'value': 'Simples'},
                        {'label': 'Complexas', 'value': 'Complexa'}
                    ],
                    value='Todas'
                )
            ], style={'width': '33%'})
        ])
    ]),

    # Gráfico de Dispersão
    dcc.Graph(id='scatter-plot'),

    # Tabela de Dados
    html.Hr(),
    html.H3("Detalhes das Transações Selecionadas"),
    dash_table.DataTable(
        id='data-table',
        columns=[{"name": i, "id": i} for i in df_final.columns],
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_header={'backgroundColor': '#003366', 'color': 'white', 'fontWeight': 'bold'},
        style_cell={'textAlign': 'left', 'padding': '5px'},
        sort_action="native",
        filter_action="native",
    )
])

# ==============================================================================
# 4. CALLBACKS - LÓGICA INTERATIVA DO DASHBOARD
# ==============================================================================
@app.callback(
    Output('scatter-plot', 'figure'),
    Output('data-table', 'data'),
    Output('data-table', 'columns'),
    Input('filtro-anomalia', 'value'),
    Input('filtro-complexidade', 'value')
)
def update_dashboard(selected_anomalia, selected_complexidade):
    """
    Atualiza o gráfico de dispersão e a tabela de dados com base nos filtros selecionados.
    """
    filtered_df = df_final.copy()

    # Aplicar filtro de anomalia
    if selected_anomalia != 'Todas':
        filtered_df = filtered_df[filtered_df['anomalia_detectada'] == selected_anomalia]

    # Aplicar filtro de complexidade
    if selected_complexidade != 'Todas':
        filtered_df = filtered_df[filtered_df['complexidade'] == selected_complexidade]

    # Criar gráfico de dispersão
    fig = px.scatter(
        filtered_df,
        x="valor",
        y="desvio_media_historica",
        color="anomalia_detectada",
        hover_data=['id_transacao', 'data_hora', 'tipo_transacao', 'categoria', 'id_entidade', 'complexidade', 'modelo_deteccao'],
        title="Transações por Valor vs. Desvio da Média (Coloridas por Anomalia)"
    )

    # Preparar dados para a tabela
    table_data = filtered_df.to_dict('records')
    table_columns = [{"name": i, "id": i} for i in filtered_df.columns]

    return fig, table_data, table_columns

# ==============================================================================
# 5. EXECUTAR A APLICAÇÃO DASH
# ==============================================================================
# Para executar no Colab, use jupyter_mode=True
if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode=True)

Bibliotecas importadas com sucesso.

--- A iniciar a preparação completa dos dados para o dashboard... ---
--- A simular dados de exemplo... ---
!! Lembre-se de substituir este bloco pelos seus dados reais. !!
--- Engenharia de Atributos... ---
--- Definindo a Complexidade... ---
--- Treinando e prevendo com os modelos... ---
68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
--- Consolidando resultados... ---
Preparação dos dados concluída.


ValueError: Cannot convert '('f', 'i', 'l', 't', 'r', 'o', '-', 'a', 'n', 'o', 'm', 'a', 'l', 'i', 'a')' to a shape. Found invalid entry 'f' of type '<class 'str'>'. 

In [ ]:
%pip install dash pandas numpy scikit-learn tensorflow plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 49.0 MB/s eta 0:00:00


In [ ]:
# ==============================================================================
# 1. SETUP - IMPORTAÇÃO DAS BIBLIOTECAS
# ==============================================================================
import dash
from dash import dcc, html, dash_table
from dash.dependencies import Input as DashInput, Output
import plotly.express as px
import pandas as pd
import numpy as np

# Importações para o pipeline de Machine Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

print("Bibliotecas importadas com sucesso.")

# ==============================================================================
# 2. FUNÇÃO DO PIPELINE DE DADOS E MODELAGEM
# (Esta função encapsula todo o processo que definimos anteriormente)
# ==============================================================================

def preparar_dados_para_dashboard():
    """
    Executa o pipeline completo: simulação de dados, engenharia de atributos,
    modelagem e predição. Retorna um DataFrame final pronto para o dashboard.
    """
    print("\n--- A iniciar a preparação completa dos dados para o dashboard... ---")

    # --- Simulação de Dados (SUBSTITUA PELOS SEUS DADOS REAIS) ---
    print("--- A simular dados de exemplo... ---")
    num_registos = 10000
    data = {
        'id_transacao': range(num_registos),
        'data_hora': pd.to_datetime(np.random.choice(pd.date_range('2024-01-01', '2025-01-01', freq='h'), num_registos)),
        'valor': np.random.lognormal(mean=8, sigma=1.5, size=num_registos).round(2),
        'tipo_transacao': np.random.choice(['Pagamento', 'Recebimento', 'Transferência'], size=num_registos),
        'categoria': np.random.choice(['Fornecedores', 'Salários', 'Marketing', 'Diversos', 'Outros'], size=num_registos, p=[0.4, 0.2, 0.2, 0.1, 0.1]),
        'id_entidade': np.random.randint(1, 50, size=num_registos)
    }
    df = pd.DataFrame(data)
    df.loc[df.sample(frac=0.02).index, 'valor'] *= np.random.uniform(5, 10)
    df.loc[df.sample(frac=0.01).index, 'valor'] *= -1
    df.loc[df.sample(frac=0.01).index, 'data_hora'] += pd.to_timedelta(np.random.randint(21, 23), unit='h')
    print("!! Lembre-se de substituir este bloco pelos seus dados reais. !!")

    # --- Engenharia de Atributos ---
    print("--- Engenharia de Atributos... ---")
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    mediana_por_entidade = df.groupby('id_entidade')['valor'].transform('median')
    df['desvio_mediana_historica'] = (df['valor'] - mediana_por_entidade) / (mediana_por_entidade + 1e-6)
    df['hora_incomum'] = ((df['data_hora'].dt.hour < 7) | (df['data_hora'].dt.hour > 20)).astype(int)
    df['dia_fim_de_semana'] = (df['data_hora'].dt.dayofweek >= 5).astype(int)
    df['arredondamento_valor'] = (df['valor'] > 1000) & (df['valor'] % 1000 == 0).astype(int)
    df = df.sort_values(by=['id_entidade', 'data_hora'])
    diferenca_tempo = df.groupby('id_entidade')['data_hora'].diff().dt.total_seconds()
    df['sequencia_rapida'] = (diferenca_tempo < 60).astype(int)
    df.fillna(0, inplace=True)

    # --- Definição da Complexidade ---
    print("--- Definindo a Complexidade... ---")
    media_geral = df['valor'].mean()
    std_geral = df['valor'].std()
    criterio_1 = df['valor'] > (media_geral + 3 * std_geral)
    criterio_2 = df['categoria'].isin(['Diversos', 'Outros'])
    criterio_3 = df['valor'] < 0
    df['complexidade'] = np.where((criterio_1 | criterio_2 | criterio_3), 'Complexa', 'Simples')

    # --- Pré-processamento e Divisão ---
    features_para_modelo = df.select_dtypes(include=np.number).columns.drop(['id_transacao', 'id_entidade'])
    X = df[features_para_modelo]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    df_scaled = pd.DataFrame(X_scaled, columns=features_para_modelo, index=df.index)

    # Adicionar colunas originais necessárias para a lógica e o resultado final
    df_scaled[['complexidade', 'id_transacao']] = df[['complexidade', 'id_transacao']]

    # --- Treino e Predição ---
    print("--- Treinando e prevendo com os modelos... ---")
    # Isolation Forest
    df_simples = df_scaled[df_scaled['complexidade'] == 'Simples'].drop(['complexidade', 'id_transacao'], axis=1)
    iso_forest = IsolationForest(n_estimators=200, contamination=0.01, random_state=42, n_jobs=-1)
    iso_forest.fit(df_simples)
    predicoes_if = iso_forest.predict(df_simples)

    # Autoencoder
    df_complexas = df_scaled[df_scaled['complexidade'] == 'Complexa'].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_complexas.empty:
        input_dim = df_complexas.shape[1]
        input_layer = Input(shape=(input_dim,))
        encoder = Dense(16, activation='relu')(input_layer)
        encoder = Dense(8, activation='relu')(encoder)
        decoder = Dense(16, activation='relu')(encoder)
        decoder = Dense(input_dim, activation='linear')(decoder)
        autoencoder = Model(inputs=input_layer, outputs=decoder)
        autoencoder.compile(optimizer='adam', loss='mean_squared_error')
        autoencoder.fit(df_complexas, df_complexas, epochs=50, batch_size=32, verbose=0)
        reconstrucoes = autoencoder.predict(df_complexas)
        mse = np.mean(np.power(df_complexas - reconstrucoes, 2), axis=1)
        threshold = np.quantile(mse, 0.95)
        predicoes_ae = np.where(mse > threshold, -1, 1)
    else:
        predicoes_ae = np.array([])


    # --- Consolidação dos Resultados ---
    print("--- Consolidando resultados... ---")
    df['anomalia_detectada'] = 'Normal'
    df['modelo_deteccao'] = 'N/A'

    indices_simples = df_simples.index
    df.loc[indices_simples, 'anomalia_detectada'] = np.where(predicoes_if == -1, 'Anómala', 'Normal')
    df.loc[indices_simples, 'modelo_deteccao'] = 'Isolation Forest'

    if not df_complexas.empty:
        indices_complexas = df_complexas.index
        df.loc[indices_complexas, 'anomalia_detectada'] = np.where(predicoes_ae == -1, 'Anómala', 'Normal')
        df.loc[indices_complexas, 'modelo_deteccao'] = 'Autoencoder'

    print("Preparação dos dados concluída.")
    return df.sort_index()

# Executar a função para obter o DataFrame final
df_final = preparar_dados_para_dashboard()

# ==============================================================================
# 3. INICIALIZAÇÃO E LAYOUT DA APLICAÇÃO DASH
# ==============================================================================
app = dash.Dash(__name__)
app.title = "Dashboard de Auditoria Financeira"

app.layout = html.Div(style={'fontFamily': 'Arial, sans-serif', 'padding': '20px'}, children=[
    html.H1("Dashboard de Auditoria de Anomalias Financeiras", style={'textAlign': 'center', 'color': '#003366'}),
    html.H2("Caso de Estudo: System Business Development (SBD)", style={'textAlign': 'center', 'color': '#505050'}),

    html.Div(className='control-panel', style={'backgroundColor': '#f2f2f2', 'padding': '20px', 'borderRadius': '5px', 'marginBottom': '20px'}, children=[
        html.H4("Filtros de Análise", style={'marginTop': 0}),
        html.Div(className='filters', style={'display': 'flex', 'gap': '40px'}, children=[
            html.Div(children=[
                html.Label("Estado da Anomalia:"),
                dcc.Dropdown(
                    id='filtro-anomalia',
                    options=[
                        {'label': 'Todas', 'value': 'Todas'},
                        {'label': 'Anómalas', 'value': 'Anómala'},
                        {'label': 'Normais', 'value': 'Normal'}
                    ],
                    value='Todas'
                )
            ], style={'width': '33%'}),

            html.Div(children=[
                html.Label("Complexidade da Transação:"),
                dcc.Dropdown(
                    id='filtro-complexidade',
                    options=[
                        {'label': 'Todas', 'value': 'Todas'},
                        {'label': 'Simples', 'value': 'Simples'},
                        {'label': 'Complexas', 'value': 'Complexa'}
                    ],
                    value='Todas'
                )
            ], style={'width': '33%'})
        ])
    ]),

    # Gráfico de Dispersão
    dcc.Graph(id='scatter-plot'),

    # Tabela de Dados
    html.Hr(),
    html.H3("Detalhes das Transações Selecionadas"),
    dash_table.DataTable(
        id='data-table',
        columns=[{"name": i, "id": i} for i in df_final.columns],
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_header={'backgroundColor': '#003366', 'color': 'white', 'fontWeight': 'bold'},
        style_cell={'textAlign': 'left', 'padding': '5px'},
        sort_action="native",
        filter_action="native",
    )
])

# ==============================================================================
# 4. CALLBACKS - LÓGICA INTERATIVA DO DASHBOARD
# ==============================================================================
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('data-table', 'data'),
     Output('data-table', 'columns')],
    [DashInput('filtro-anomalia', 'value'), # Modificado aqui
     DashInput('filtro-complexidade', 'value')] # E aqui
)
def update_dashboard(selected_anomalia, selected_complexidade):
    """
    Atualiza o gráfico de dispersão e a tabela de dados com base nos filtros selecionados.
    """
    filtered_df = df_final.copy()

    # Aplicar filtro de anomalia
    if selected_anomalia != 'Todas':
        filtered_df = filtered_df[filtered_df['anomalia_detectada'] == selected_anomalia]

    # Aplicar filtro de complexidade
    if selected_complexidade != 'Todas':
        filtered_df = filtered_df[filtered_df['complexidade'] == selected_complexidade]

    # Criar gráfico de dispersão
    fig = px.scatter(
        filtered_df,
        x="valor",
        y="desvio_media_historica",
        color="anomalia_detectada",
        hover_data=['id_transacao', 'data_hora', 'tipo_transacao', 'categoria', 'id_entidade', 'complexidade', 'modelo_deteccao'],
        title="Transações por Valor vs. Desvio da Média (Coloridas por Anomalia)"
    )

    # Preparar dados para a tabela
    table_data = filtered_df.to_dict('records')
    table_columns = [{"name": i, "id": i} for i in filtered_df.columns]

    return fig, table_data, table_columns

# ==============================================================================
# 5. EXECUTAR A APLICAÇÃO DASH
# ==============================================================================
# Para executar no Colab, use jupyter_mode=True
if __name__ == '__main__':
    app.run_server(debug=True, jupyter_mode=True)

Bibliotecas importadas com sucesso.

--- A iniciar a preparação completa dos dados para o dashboard... ---
--- A simular dados de exemplo... ---
!! Lembre-se de substituir este bloco pelos seus dados reais. !!
--- Engenharia de Atributos... ---
--- Definindo a Complexidade... ---
--- Treinando e prevendo com os modelos... ---
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
--- Consolidando resultados... ---
Preparação dos dados concluída.


ValueError: Cannot convert '('f', 'i', 'l', 't', 'r', 'o', '-', 'a', 'n', 'o', 'm', 'a', 'l', 'i', 'a')' to a shape. Found invalid entry 'f' of type '<class 'str'>'. 

In [ ]:
# ==============================================================================
# 0. SETUP PARA GOOGLE COLAB - INSTALAÇÃO DE BIBLIOTECAS
# ==============================================================================
# !pip install jupyter-dash -q # Jupyter-dash is not needed with newer Dash versions

# ==============================================================================
# 1. SETUP - IMPORTAÇÃO DAS BIBLIOTECAS (COM CORREÇÃO DE CONFLITO)
# ==============================================================================
import dash # Use dash directly
from dash import dcc, html, dash_table
from dash.dependencies import Input as DashInput, Output
import plotly.express as px
import pandas as pd
import numpy as np

# Bibliotecas de Machine Learning e Pré-processamento
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest

# Bibliotecas para o Autoencoder (Deep Learning)
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

print("Bibliotecas importadas com sucesso.")

# ==============================================================================
# 2. FUNÇÃO DO PIPELINE DE DADOS E MODELAGEM
# ==============================================================================
def preparar_dados_para_dashboard():
    """
    Executa o pipeline completo: simulação de dados, engenharia de atributos,
    modelagem e predição. Retorna um DataFrame final pronto para o dashboard.
    """
    print("\n--- A iniciar a preparação completa dos dados para o dashboard... ---")

    # --- Simulação de Dados (SUBSTITUA PELOS SEUS DADOS REAIS) ---
    print("--- A simular dados de exemplo... ---")
    num_registos = 10000
    data = {
        'id_transacao': range(num_registos),
        'data_hora': pd.to_datetime(np.random.choice(pd.date_range('2024-01-01', '2025-01-01', freq='h'), num_registos)),
        'valor': np.random.lognormal(mean=8, sigma=1.5, size=num_registos).round(2),
        'tipo_transacao': np.random.choice(['Pagamento', 'Recebimento', 'Transferência'], size=num_registos),
        'categoria': np.random.choice(['Fornecedores', 'Salários', 'Marketing', 'Diversos', 'Outros'], size=num_registos, p=[0.4, 0.2, 0.2, 0.1, 0.1]),
        'id_entidade': np.random.randint(1, 50, size=num_registos)
    }
    df = pd.DataFrame(data)
    df.loc[df.sample(frac=0.02).index, 'valor'] *= np.random.uniform(5, 10)
    df.loc[df.sample(frac=0.01).index, 'valor'] *= -1
    df.loc[df.sample(frac=0.01).index, 'data_hora'] += pd.to_timedelta(np.random.randint(21, 23), unit='h')
    print("!! Lembre-se de substituir este bloco pelos seus dados reais. !!")

    # --- Engenharia de Atributos ---
    print("--- Engenharia de Atributos... ---")
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    mediana_por_entidade = df.groupby('id_entidade')['valor'].transform('median')
    df['desvio_mediana_historica'] = (df['valor'] - mediana_por_entidade) / (mediana_por_entidade + 1e-6)
    df['hora_incomum'] = ((df['data_hora'].dt.hour < 7) | (df['data_hora'].dt.hour > 20)).astype(int)
    df['dia_fim_de_semana'] = (df['data_hora'].dt.dayofweek >= 5).astype(int)
    df['arredondamento_valor'] = (df['valor'] > 1000) & (df['valor'] % 1000 == 0).astype(int)
    df = df.sort_values(by=['id_entidade', 'data_hora'])
    diferenca_tempo = df.groupby('id_entidade')['data_hora'].diff().dt.total_seconds()
    df['sequencia_rapida'] = (diferenca_tempo < 60).astype(int)
    df.fillna(0, inplace=True)

    # --- Definição da Complexidade ---
    print("--- Definindo a Complexidade... ---")
    media_geral = df['valor'].mean()
    std_geral = df['valor'].std()
    criterio_1 = df['valor'] > (media_geral + 3 * std_geral)
    criterio_2 = df['categoria'].isin(['Diversos', 'Outros'])
    criterio_3 = df['valor'] < 0
    df['complexidade'] = np.where((criterio_1 | criterio_2 | criterio_3), 'Complexa', 'Simples')

    # --- Pré-processamento e Divisão ---
    features_para_modelo = df.select_dtypes(include=np.number).columns.drop(['id_transacao', 'id_entidade'])
    X = df[features_para_modelo]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    df_scaled = pd.DataFrame(X_scaled, columns=features_para_modelo, index=df.index)
    df_scaled[['complexidade', 'id_transacao']] = df[['complexidade', 'id_transacao']]

    # --- Treino e Predição ---
    print("--- Treinando e prevendo com os modelos... ---")
    # Isolation Forest
    df_simples = df_scaled[df_scaled['complexidade'] == 'Simples'].drop(['complexidade', 'id_transacao'], axis=1)
    iso_forest = IsolationForest(n_estimators=200, contamination=0.01, random_state=42, n_jobs=-1)
    iso_forest.fit(df_simples)
    predicoes_if = iso_forest.predict(df_simples)

    # Autoencoder
    df_complexas = df_scaled[df_scaled['complexidade'] == 'Complexa'].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_complexas.empty:
        input_dim = df_complexas.shape[1]
        input_layer = Input(shape=(input_dim,))
        encoder = Dense(16, activation='relu')(input_layer)
        encoder = Dense(8, activation='relu')(encoder)
        decoder = Dense(16, activation='relu')(encoder)
        decoder = Dense(input_dim, activation='linear')(decoder)
        autoencoder = Model(inputs=input_layer, outputs=decoder)
        autoencoder.compile(optimizer='adam', loss='mean_squared_error')
        autoencoder.fit(df_complexas, df_complexas, epochs=50, batch_size=32, verbose=0)
        reconstrucoes = autoencoder.predict(df_complexas)
        mse = np.mean(np.power(df_complexas - reconstrucoes, 2), axis=1)
        threshold = np.quantile(mse, 0.95)
        predicoes_ae = np.where(mse > threshold, -1, 1)
    else:
        predicoes_ae = np.array([])

    # --- Consolidação dos Resultados ---
    print("--- Consolidando resultados... ---")
    df['anomalia_detectada'] = 'Normal'
    df['modelo_deteccao'] = 'N/A'

    indices_simples = df_simples.index
    df.loc[indices_simples, 'anomalia_detectada'] = np.where(predicoes_if == -1, 'Anómala', 'Normal')
    df.loc[indices_simples, 'modelo_deteccao'] = 'Isolation Forest'

    if not df_complexas.empty:
        indices_complexas = df_complexas.index
        df.loc[indices_complexas, 'anomalia_detectada'] = np.where(predicoes_ae == -1, 'Anómala', 'Normal')
        df.loc[indices_complexas, 'modelo_deteccao'] = 'Autoencoder'

    print("Preparação dos dados concluída.")
    return df.sort_index()

# Executar a função para obter o DataFrame final
df_final = preparar_dados_para_dashboard()

# ==============================================================================
# 3. INICIALIZAÇÃO E LAYOUT DA APLICAÇÃO DASH
# ==============================================================================
app = dash.Dash(__name__) # Use dash.Dash directly
app.title = "Dashboard de Auditoria Financeira"

app.layout = html.Div(style={'fontFamily': 'Arial, sans-serif', 'padding': '20px'}, children=[
    html.H1("Dashboard de Auditoria de Anomalias Financeiras", style={'textAlign': 'center', 'color': '#003366'}),
    html.H2("Caso de Estudo: System Business Development (SBD)", style={'textAlign': 'center', 'color': '#505050'}),

    html.Div(className='control-panel', style={'backgroundColor': '#f2f2f2', 'padding': '20px', 'borderRadius': '5px', 'marginBottom': '20px'}, children=[
        html.H4("Filtros de Análise", style={'marginTop': 0}),
        html.Div(className='filters', style={'display': 'flex', 'gap': '40px'}, children=[
            html.Div(children=[
                html.Label("Estado da Anomalia:"),
                dcc.Dropdown(
                    id='filtro-anomalia',
                    options=[
                        {'label': 'Todas', 'value': 'Todas'},
                        {'label': 'Anómalas', 'value': 'Anómala'},
                        {'label': 'Normais', 'value': 'Normal'}
                    ],
                    value='Todas'
                )
            ], style={'width': '33%'}),

            html.Div(children=[
                html.Label("Complexidade da Transação:"),
                dcc.Dropdown(
                    id='filtro-complexidade',
                    options=[
                        {'label': 'Todas', 'value': 'Todas'},
                        {'label': 'Simples', 'value': 'Simples'},
                        {'label': 'Complexas', 'value': 'Complexa'}
                    ],
                    value='Todas'
                )
            ], style={'width': '33%'})
        ])
    ]),

    dcc.Graph(id='scatter-plot'),

    html.Hr(),
    html.H3("Detalhes das Transações Selecionadas"),
    dash_table.DataTable(
        id='data-table',
        columns=[{"name": i, "id": i} for i in df_final.columns],
        page_size=10,
        style_table={'overflowX': 'auto'},
        style_header={'backgroundColor': '#003366', 'color': 'white', 'fontWeight': 'bold'},
        style_cell={'textAlign': 'left', 'padding': '5px'},
        sort_action="native",
        filter_action="native",
    )
])

# ==============================================================================
# 4. CALLBACKS - LÓGICA INTERATIVA DO DASHBOARD (COM CORREÇÃO)
# ==============================================================================
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('data-table', 'data'),
     Output('data-table', 'columns')],
    [DashInput('filtro-anomalia', 'value'),
     DashInput('filtro-complexidade', 'value')]
)
def update_dashboard(status_anomalia, tipo_complexidade):
    dff = df_final.copy()

    if status_anomalia != 'Todas':
        dff = dff[dff['anomalia_detectada'] == status_anomalia]

    if tipo_complexidade != 'Todas':
        dff = dff[dff['complexidade'] == tipo_complexidade]

    fig = px.scatter(
        dff, x='data_hora', y='valor',
        color='anomalia_detectada', size='valor',
        hover_data=['id_transacao', 'categoria', 'complexidade', 'modelo_deteccao'],
        color_discrete_map={'Anómala': '#FF4136', 'Normal': '#0074D9'},
        title="Visualização de Transações Financeiras",
        template='plotly_white'
    )
    fig.update_layout(transition_duration=500, xaxis_title="Data da Transação", yaxis_title="Valor (MZN)")

    colunas_tabela = [{"name": i, "id": i} for i in dff.columns]
    dados_tabela = dff.to_dict('records')

    return fig, dados_tabela, colunas_tabela

# ==============================================================================
# 5. EXECUÇÃO DA APLICAÇÃO NO COLAB
# ==============================================================================
if __name__ == '__main__':
    app.run(mode='inline') # Use mode='inline' for Colab

Bibliotecas importadas com sucesso.

--- A iniciar a preparação completa dos dados para o dashboard... ---
--- A simular dados de exemplo... ---
!! Lembre-se de substituir este bloco pelos seus dados reais. !!
--- Engenharia de Atributos... ---
--- Definindo a Complexidade... ---
--- Treinando e prevendo com os modelos... ---
67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
--- Consolidando resultados... ---
Preparação dos dados concluída.
Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__'
 * Debug mode: off


TypeError: run_simple() got an unexpected keyword argument 'mode'

In [ ]:
# ==============================================================================
# 0. SETUP PARA GOOGLE COLAB - INSTALAÇÃO DE BIBLIOTECAS
# ==============================================================================
!pip install jupyter-dash dash-bootstrap-components -q

# ==============================================================================
# 1. SETUP - IMPORTAÇÃO DAS BIBLIOTECAS (COM CORREÇÃO DE CONFLITO)
# ==============================================================================
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
from dash.dependencies import Input as DashInput, Output
import plotly.express as px
import pandas as pd
import numpy as np

# Bibliotecas de Machine Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

print("Bibliotecas importadas com sucesso.")

# ==============================================================================
# 2. FUNÇÃO DO PIPELINE DE DADOS E MODELAGEM
# ==============================================================================
def preparar_dados_para_dashboard():
    """
    Executa o pipeline completo: simulação, engenharia de atributos, modelagem
    e predição. Retorna um DataFrame final pronto para o dashboard.
    """
    print("\n--- A iniciar a preparação completa dos dados... ---")

    # --- Simulação de Dados (SUBSTITUA PELOS SEUS DADOS REAIS) ---
    print("--- A simular dados de exemplo... ---")
    num_registos = 10000
    data = {
        'id_transacao': range(num_registos),
        'data_hora': pd.to_datetime(np.random.choice(pd.date_range('2024-01-01', '2025-01-01', freq='h'), num_registos)),
        'valor': np.random.lognormal(mean=8, sigma=1.5, size=num_registos).round(2),
        'tipo_transacao': np.random.choice(['Pagamento', 'Recebimento', 'Transferência'], size=num_registos),
        'categoria': np.random.choice(['Conta Corrente', 'Salários', 'Marketing', 'Diversos', 'Impostos'], size=num_registos, p=[0.4, 0.2, 0.2, 0.1, 0.1]),
        'id_entidade': np.random.randint(1, 50, size=num_registos)
    }
    df = pd.DataFrame(data)
    df.loc[df.sample(frac=0.02).index, 'valor'] *= np.random.uniform(5, 10)
    df.loc[df.sample(frac=0.01).index, 'valor'] *= -1
    df.loc[df.sample(frac=0.01).index, 'data_hora'] += pd.to_timedelta(np.random.randint(21, 23), unit='h')

    # --- Engenharia de Atributos ---
    print("--- Engenharia de Atributos... ---")
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    # Adicionando 'dia_do_ano' para o gráfico
    df['dia_do_ano'] = df['data_hora'].dt.dayofyear
    mediana_por_entidade = df.groupby('id_entidade')['valor'].transform('median')
    df['desvio_mediana_historica'] = (df['valor'] - mediana_por_entidade) / (mediana_por_entidade + 1e-6)
    df['hora_incomum'] = ((df['data_hora'].dt.hour < 7) | (df['data_hora'].dt.hour > 20)).astype(int)
    df['dia_fim_de_semana'] = (df['data_hora'].dt.dayofweek >= 5).astype(int)
    df['arredondamento_valor'] = (df['valor'] > 1000) & (df['valor'] % 1000 == 0).astype(int)
    df = df.sort_values(by=['id_entidade', 'data_hora'])
    diferenca_tempo = df.groupby('id_entidade')['data_hora'].diff().dt.total_seconds()
    df['sequencia_rapida'] = (diferenca_tempo < 60).astype(int)
    df.fillna(0, inplace=True)

    # --- Definição da Complexidade ---
    print("--- Definindo a Complexidade... ---")
    media_geral = df['valor'].mean()
    std_geral = df['valor'].std()
    criterio_1 = df['valor'] > (media_geral + 3 * std_geral)
    criterio_2 = df['categoria'].isin(['Diversos', 'Outros'])
    criterio_3 = df['valor'] < 0
    df['complexidade'] = np.where((criterio_1 | criterio_2 | criterio_3), 'Complexa', 'Simples')

    # --- Pré-processamento e Divisão ---
    features_para_modelo = df.select_dtypes(include=np.number).columns.drop(['id_transacao', 'id_entidade'])
    X = df[features_para_modelo]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    df_scaled = pd.DataFrame(X_scaled, columns=features_para_modelo, index=df.index)
    df_scaled[['complexidade', 'id_transacao']] = df[['complexidade', 'id_transacao']]

    # --- Treino e Predição ---
    print("--- Treinando e prevendo com os modelos... ---")
    df['anomalia_detectada'] = 'Normal'
    df['modelo_deteccao'] = 'N/A'

    # Isolation Forest
    indices_simples = df_scaled[df_scaled['complexidade'] == 'Simples'].index
    df_simples = df_scaled.loc[indices_simples].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_simples.empty:
        iso_forest = IsolationForest(n_estimators=200, contamination=0.01, random_state=42, n_jobs=-1)
        iso_forest.fit(df_simples)
        predicoes_if = iso_forest.predict(df_simples)
        df.loc[indices_simples, 'anomalia_detectada'] = np.where(predicoes_if == -1, 'Anómala', 'Normal')
        df.loc[indices_simples, 'modelo_deteccao'] = 'Isolation Forest'

    # Autoencoder
    indices_complexas = df_scaled[df_scaled['complexidade'] == 'Complexa'].index
    df_complexas = df_scaled.loc[indices_complexas].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_complexas.empty:
        input_dim = df_complexas.shape[1]
        input_layer = Input(shape=(input_dim,))
        encoder = Dense(16, activation='relu')(input_layer)
        encoder = Dense(8, activation='relu')(encoder)
        decoder = Dense(16, activation='relu')(encoder)
        decoder = Dense(input_dim, activation='linear')(decoder)
        autoencoder = Model(inputs=input_layer, outputs=decoder)
        autoencoder.compile(optimizer='adam', loss='mean_squared_error')
        autoencoder.fit(df_complexas, df_complexas, epochs=50, batch_size=32, verbose=0)
        reconstrucoes = autoencoder.predict(df_complexas)
        mse = np.mean(np.power(df_complexas.values - reconstrucoes, 2), axis=1)
        threshold = np.quantile(mse, 0.95)
        predicoes_ae = np.where(mse > threshold, -1, 1)
        df.loc[indices_complexas, 'anomalia_detectada'] = np.where(predicoes_ae == -1, 'Anómala', 'Normal')
        df.loc[indices_complexas, 'modelo_deteccao'] = 'Autoencoder'

    # Renomear 'categoria' para 'conta' para corresponder à interface do dashboard
    df.rename(columns={'categoria': 'conta'}, inplace=True)

    print("Preparação dos dados concluída.")
    return df.sort_index()

# ==============================================================================
# 3. EXECUÇÃO DO PIPELINE E PREPARAÇÃO DOS DADOS FINAIS
# ==============================================================================
df_final = preparar_dados_para_dashboard()

# ==============================================================================
# 4. INICIALIZAÇÃO E LAYOUT DA APLICAÇÃO DASH (VERSÃO INTEGRADA)
# ==============================================================================
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Dashboard de Auditoria Financeira"

app.layout = dbc.Container([
    html.H1("Dashboard de Auditoria de Anomalias Financeiras", className="my-4 text-center"),
    html.H2("Caso de Estudo: SBD (com Pipeline Otimizado)", className="text-center text-muted mb-4"),

    dbc.Row([
        dbc.Col([
            html.Label("Conta:"),
            dcc.Dropdown(
                id="conta-dropdown",
                options=[{"label": c, "value": c} for c in ["Todos"] + sorted(df_final["conta"].unique())],
                value="Todos"
            )
        ], md=4),

        dbc.Col([
            html.Label("Tipo de Complexidade:"),
            dcc.Dropdown(
                id="tipo-dropdown",
                options=[{"label": c, "value": c} for c in ["Todos"] + df_final["complexidade"].unique().tolist()],
                value="Todos"
            )
        ], md=4),

        dbc.Col([
            html.Label("Estado da Anomalia:"),
            dcc.Dropdown(
                id="anomalia-dropdown",
                options=[
                    {"label": "Todas", "value": "Todas"},
                    {"label": "Apenas Anômalas", "value": "Anómalas"},
                    {"label": "Apenas Normais", "value": "Normais"}
                ],
                value="Todas"
            )
        ], md=4)
    ], className="mb-4"),

    dcc.Graph(id="scatter-graph"),

    dash_table.DataTable(
        id="table",
        columns=[{"name": col, "id": col} for col in ["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]],
        page_size=10,
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left"},
        style_header={'backgroundColor': '#003366', 'color': 'white', 'fontWeight': 'bold'},
        sort_action="native",
        filter_action="native",
    )
], fluid=True)

# ==============================================================================
# 5. CALLBACK - LÓGICA INTERATIVA DO DASHBOARD
# ==============================================================================
@app.callback(
    [Output("scatter-graph", "figure"),
     Output("table", "data")],
    [DashInput("conta-dropdown", "value"),
     DashInput("tipo-dropdown", "value"),
     DashInput("anomalia-dropdown", "value")]
)
def actualizar_dashboard(conta, tipo, anomalia):
    df_filtrado = df_final.copy()

    if conta != "Todos":
        df_filtrado = df_filtrado[df_filtrado["conta"] == conta]
    if tipo != "Todos":
        df_filtrado = df_filtrado[df_filtrado["complexidade"] == tipo]

    # Lógica de filtro para anomalias ajustada para os novos valores de texto
    if anomalia == "Anómalas":
        df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Anómala"]
    elif anomalia == "Normais":
        df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Normal"]

    # Criar o gráfico
    fig = px.scatter(
        df_filtrado, x="valor", y="dia_do_ano",
        color="anomalia_detectada",
        hover_data=['conta', 'complexidade', 'modelo_deteccao'],
        color_discrete_map={'Anómala': '#FF4136', 'Normal': '#0074D9'},
        title="Visualização de Anomalias (Valor vs. Dia do Ano)",
        labels={"valor": "Valor da Transação (MZN)", "dia_do_ano": "Dia do Ano"}
    )
    fig.update_layout(transition_duration=500)

    # Preparar dados para a tabela
    data_tabela = df_filtrado[["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]].to_dict("records")

    return fig, data_tabela

# ==============================================================================
# 6. EXECUÇÃO DA APLICAÇÃO NO COLAB
# ==============================================================================
if __name__ == "__main__":
    app.run_server(mode='inline')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.7/203.7 kB 11.7 MB/s eta 0:00:00
Bibliotecas importadas com sucesso.

--- A iniciar a preparação completa dos dados... ---
--- A simular dados de exemplo... ---
--- Engenharia de Atributos... ---
--- Definindo a Complexidade... ---
--- Treinando e prevendo com os modelos... ---
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Preparação dos dados concluída.


/usr/local/lib/python3.11/dist-packages/dash/dash.py:634: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



AttributeError: 'super' object has no attribute 'run_server'

In [ ]:
# ==============================================================================
# 0. SETUP PARA GOOGLE COLAB - INSTALAÇÃO DE BIBLIOTECAS
# ==============================================================================
!pip install dash dash-bootstrap-components plotly pandas scikit-learn tensorflow -q

# ==============================================================================
# 1. SETUP - IMPORTAÇÃO DAS BIBLIOTECAS (COM CORREÇÃO FINAL)
# ==============================================================================
from dash import Dash # CORREÇÃO: Usar a biblioteca Dash principal
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
from dash.dependencies import Input as DashInput, Output
import plotly.express as px
import pandas as pd
import numpy as np

# Bibliotecas de Machine Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

print("Bibliotecas importadas com sucesso.")

# ==============================================================================
# 2. FUNÇÃO DO PIPELINE DE DADOS E MODELAGEM
# ==============================================================================
def preparar_dados_para_dashboard():
    """
    Executa o pipeline completo: simulação, engenharia de atributos, modelagem
    e predição. Retorna um DataFrame final pronto para o dashboard.
    """
    print("\n--- A iniciar a preparação completa dos dados... ---")

    # --- Simulação de Dados (SUBSTITUA PELOS SEUS DADOS REAIS) ---
    print("--- A simular dados de exemplo... ---")
    num_registos = 10000
    data = {
        'id_transacao': range(num_registos),
        'data_hora': pd.to_datetime(np.random.choice(pd.date_range('2024-01-01', '2025-01-01', freq='h'), num_registos)),
        'valor': np.random.lognormal(mean=8, sigma=1.5, size=num_registos).round(2),
        'tipo_transacao': np.random.choice(['Pagamento', 'Recebimento', 'Transferência'], size=num_registos),
        'categoria': np.random.choice(['Conta Corrente', 'Salários', 'Marketing', 'Diversos', 'Impostos'], size=num_registos, p=[0.4, 0.2, 0.2, 0.1, 0.1]),
        'id_entidade': np.random.randint(1, 50, size=num_registos)
    }
    df = pd.DataFrame(data)
    df.loc[df.sample(frac=0.02).index, 'valor'] *= np.random.uniform(5, 10)
    df.loc[df.sample(frac=0.01).index, 'valor'] *= -1
    df.loc[df.sample(frac=0.01).index, 'data_hora'] += pd.to_timedelta(np.random.randint(21, 23), unit='h')

    # --- Engenharia de Atributos ---
    print("--- Engenharia de Atributos... ---")
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    df['dia_do_ano'] = df['data_hora'].dt.dayofyear
    mediana_por_entidade = df.groupby('id_entidade')['valor'].transform('median')
    df['desvio_mediana_historica'] = (df['valor'] - mediana_por_entidade) / (mediana_por_entidade + 1e-6)
    df['hora_incomum'] = ((df['data_hora'].dt.hour < 7) | (df['data_hora'].dt.hour > 20)).astype(int)
    df['dia_fim_de_semana'] = (df['data_hora'].dt.dayofweek >= 5).astype(int)
    df['arredondamento_valor'] = (df['valor'] > 1000) & (df['valor'] % 1000 == 0).astype(int)
    df = df.sort_values(by=['id_entidade', 'data_hora'])
    diferenca_tempo = df.groupby('id_entidade')['data_hora'].diff().dt.total_seconds()
    df['sequencia_rapida'] = (diferenca_tempo < 60).astype(int)
    df.fillna(0, inplace=True)

    # --- Definição da Complexidade ---
    print("--- Definindo a Complexidade... ---")
    media_geral = df['valor'].mean()
    std_geral = df['valor'].std()
    criterio_1 = df['valor'] > (media_geral + 3 * std_geral)
    criterio_2 = df['categoria'].isin(['Diversos', 'Outros'])
    criterio_3 = df['valor'] < 0
    df['complexidade'] = np.where((criterio_1 | criterio_2 | criterio_3), 'Complexa', 'Simples')

    # --- Pré-processamento e Divisão ---
    features_para_modelo = df.select_dtypes(include=np.number).columns.drop(['id_transacao', 'id_entidade'])
    X = df[features_para_modelo]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    df_scaled = pd.DataFrame(X_scaled, columns=features_para_modelo, index=df.index)
    df_scaled[['complexidade', 'id_transacao']] = df[['complexidade', 'id_transacao']]

    # --- Treino e Predição ---
    print("--- Treinando e prevendo com os modelos... ---")
    df['anomalia_detectada'] = 'Normal'
    df['modelo_deteccao'] = 'N/A'

    # Isolation Forest
    indices_simples = df_scaled[df_scaled['complexidade'] == 'Simples'].index
    df_simples = df_scaled.loc[indices_simples].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_simples.empty:
        iso_forest = IsolationForest(n_estimators=200, contamination=0.01, random_state=42, n_jobs=-1)
        iso_forest.fit(df_simples)
        predicoes_if = iso_forest.predict(df_simples)
        df.loc[indices_simples, 'anomalia_detectada'] = np.where(predicoes_if == -1, 'Anómala', 'Normal')
        df.loc[indices_simples, 'modelo_deteccao'] = 'Isolation Forest'

    # Autoencoder
    indices_complexas = df_scaled[df_scaled['complexidade'] == 'Complexa'].index
    df_complexas = df_scaled.loc[indices_complexas].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_complexas.empty:
        input_dim = df_complexas.shape[1]
        input_layer = Input(shape=(input_dim,))
        encoder = Dense(16, activation='relu')(input_layer)
        encoder = Dense(8, activation='relu')(encoder)
        decoder = Dense(16, activation='relu')(encoder)
        decoder = Dense(input_dim, activation='linear')(decoder)
        autoencoder = Model(inputs=input_layer, outputs=decoder)
        autoencoder.compile(optimizer='adam', loss='mean_squared_error')
        autoencoder.fit(df_complexas, df_complexas, epochs=50, batch_size=32, verbose=0)
        reconstrucoes = autoencoder.predict(df_complexas)
        mse = np.mean(np.power(df_complexas.values - reconstrucoes, 2), axis=1)
        threshold = np.quantile(mse, 0.95)
        predicoes_ae = np.where(mse > threshold, -1, 1)
        df.loc[indices_complexas, 'anomalia_detectada'] = np.where(predicoes_ae == -1, 'Anómala', 'Normal')
        df.loc[indices_complexas, 'modelo_deteccao'] = 'Autoencoder'

    # Renomear 'categoria' para 'conta' para corresponder à interface do dashboard
    df.rename(columns={'categoria': 'conta'}, inplace=True)

    print("Preparação dos dados concluída.")
    return df.sort_index()

# ==============================================================================
# 3. EXECUÇÃO DO PIPELINE E PREPARAÇÃO DOS DADOS FINAIS
# ==============================================================================
df_final = preparar_dados_para_dashboard()

# ==============================================================================
# 4. INICIALIZAÇÃO E LAYOUT DA APLICAÇÃO DASH (VERSÃO CORRIGIDA PARA COLAB)
# ==============================================================================
# CORREÇÃO: Usar Dash e especificar o modo para jupyter/colab na inicialização
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], jupyter_mode="inline")
app.title = "Dashboard de Auditoria Financeira"

app.layout = dbc.Container([
    html.H1("Dashboard de Auditoria de Anomalias Financeiras", className="my-4 text-center"),
    html.H2("Caso de Estudo: SBD (com Pipeline Otimizado)", className="text-center text-muted mb-4"),

    dbc.Row([
        dbc.Col([
            html.Label("Conta:"),
            dcc.Dropdown(
                id="conta-dropdown",
                options=[{"label": c, "value": c} for c in ["Todos"] + sorted(df_final["conta"].unique())],
                value="Todos"
            )
        ], md=4),

        dbc.Col([
            html.Label("Tipo de Complexidade:"),
            dcc.Dropdown(
                id="tipo-dropdown",
                options=[{"label": c, "value": c} for c in ["Todos"] + df_final["complexidade"].unique().tolist()],
                value="Todos"
            )
        ], md=4),

        dbc.Col([
            html.Label("Estado da Anomalia:"),
            dcc.Dropdown(
                id="anomalia-dropdown",
                options=[
                    {"label": "Todas", "value": "Todas"},
                    {"label": "Apenas Anômalas", "value": "Anómalas"},
                    {"label": "Apenas Normais", "value": "Normais"}
                ],
                value="Todas"
            )
        ], md=4)
    ], className="mb-4"),

    dcc.Graph(id="scatter-graph"),

    dash_table.DataTable(
        id="table",
        columns=[{"name": col, "id": col} for col in ["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]],
        page_size=10,
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left"},
        style_header={'backgroundColor': '#003366', 'color': 'white', 'fontWeight': 'bold'},
        sort_action="native",
        filter_action="native",
    )
], fluid=True)

# ==============================================================================
# 5. CALLBACK - LÓGICA INTERATIVA DO DASHBOARD
# ==============================================================================
@app.callback(
    [Output("scatter-graph", "figure"),
     Output("table", "data")],
    [DashInput("conta-dropdown", "value"),
     DashInput("tipo-dropdown", "value"),
     DashInput("anomalia-dropdown", "value")]
)
def actualizar_dashboard(conta, tipo, anomalia):
    df_filtrado = df_final.copy()

    if conta != "Todos":
        df_filtrado = df_filtrado[df_filtrado["conta"] == conta]
    if tipo != "Todos":
        df_filtrado = df_filtrado[df_filtrado["complexidade"] == tipo]

    if anomalia == "Anómalas":
        df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Anómala"]
    elif anomalia == "Normais":
        df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Normal"]

    fig = px.scatter(
        df_filtrado, x="valor", y="dia_do_ano",
        color="anomalia_detectada",
        hover_data=['conta', 'complexidade', 'modelo_deteccao'],
        color_discrete_map={'Anómala': '#FF4136', 'Normal': '#0074D9'},
        title="Visualização de Anomalias (Valor vs. Dia do Ano)",
        labels={"valor": "Valor da Transação (MZN)", "dia_do_ano": "Dia do Ano"}
    )
    fig.update_layout(transition_duration=500)

    data_tabela = df_filtrado[["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]].to_dict("records")

    return fig, data_tabela

# ==============================================================================
# 6. EXECUÇÃO DA APLICAÇÃO NO COLAB (FORMA CORRETA)
# ==============================================================================
if __name__ == "__main__":
    # CORREÇÃO: Usar app.run() para a nova versão do Dash
    app.run(debug=True)

Bibliotecas importadas com sucesso.

--- A iniciar a preparação completa dos dados... ---
--- A simular dados de exemplo... ---
--- Engenharia de Atributos... ---
--- Definindo a Complexidade... ---
--- Treinando e prevendo com os modelos... ---
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Preparação dos dados concluída.


TypeError: Dash() got an unexpected keyword argument 'jupyter_mode'

In [ ]:
# ==============================================================================
# 0. SETUP PARA GOOGLE COLAB - INSTALAÇÃO DE BIBLIOTECAS
# ==============================================================================
!pip install dash dash-bootstrap-components plotly pandas scikit-learn tensorflow -q

# ==============================================================================
# 1. SETUP - IMPORTAÇÃO DAS BIBLIOTECAS (COM CORREÇÃO FINAL)
# ==============================================================================
from dash import Dash # CORREÇÃO: Usar a biblioteca Dash principal
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
from dash.dependencies import Input as DashInput, Output
import plotly.express as px
import pandas as pd
import numpy as np

# Bibliotecas de Machine Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

print("Bibliotecas importadas com sucesso.")

# ==============================================================================
# 2. FUNÇÃO DO PIPELINE DE DADOS E MODELAGEM
# ==============================================================================
def preparar_dados_para_dashboard():
    """
    Executa o pipeline completo: simulação, engenharia de atributos, modelagem
    e predição. Retorna um DataFrame final pronto para o dashboard.
    """
    print("\n--- A iniciar a preparação completa dos dados... ---")

    # --- Simulação de Dados (SUBSTITUA PELOS SEUS DADOS REAIS) ---
    print("--- A simular dados de exemplo... ---")
    num_registos = 10000
    data = {
        'id_transacao': range(num_registos),
        'data_hora': pd.to_datetime(np.random.choice(pd.date_range('2024-01-01', '2025-01-01', freq='h'), num_registos)),
        'valor': np.random.lognormal(mean=8, sigma=1.5, size=num_registos).round(2),
        'tipo_transacao': np.random.choice(['Pagamento', 'Recebimento', 'Transferência'], size=num_registos),
        'categoria': np.random.choice(['Conta Corrente', 'Salários', 'Marketing', 'Diversos', 'Impostos'], size=num_registos, p=[0.4, 0.2, 0.2, 0.1, 0.1]),
        'id_entidade': np.random.randint(1, 50, size=num_registos)
    }
    df = pd.DataFrame(data)
    df.loc[df.sample(frac=0.02).index, 'valor'] *= np.random.uniform(5, 10)
    df.loc[df.sample(frac=0.01).index, 'valor'] *= -1
    df.loc[df.sample(frac=0.01).index, 'data_hora'] += pd.to_timedelta(np.random.randint(21, 23), unit='h')

    # --- Engenharia de Atributos ---
    print("--- Engenharia de Atributos... ---")
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    df['dia_do_ano'] = df['data_hora'].dt.dayofyear
    mediana_por_entidade = df.groupby('id_entidade')['valor'].transform('median')
    df['desvio_mediana_historica'] = (df['valor'] - mediana_por_entidade) / (mediana_por_entidade + 1e-6)
    df['hora_incomum'] = ((df['data_hora'].dt.hour < 7) | (df['data_hora'].dt.hour > 20)).astype(int)
    df['dia_fim_de_semana'] = (df['data_hora'].dt.dayofweek >= 5).astype(int)
    df['arredondamento_valor'] = (df['valor'] > 1000) & (df['valor'] % 1000 == 0).astype(int)
    df = df.sort_values(by=['id_entidade', 'data_hora'])
    diferenca_tempo = df.groupby('id_entidade')['data_hora'].diff().dt.total_seconds()
    df['sequencia_rapida'] = (diferenca_tempo < 60).astype(int)
    df.fillna(0, inplace=True)

    # --- Definição da Complexidade ---
    print("--- Definindo a Complexidade... ---")
    media_geral = df['valor'].mean()
    std_geral = df['valor'].std()
    criterio_1 = df['valor'] > (media_geral + 3 * std_geral)
    criterio_2 = df['categoria'].isin(['Diversos', 'Outros'])
    criterio_3 = df['valor'] < 0
    df['complexidade'] = np.where((criterio_1 | criterio_2 | criterio_3), 'Complexa', 'Simples')

    # --- Pré-processamento e Divisão ---
    features_para_modelo = df.select_dtypes(include=np.number).columns.drop(['id_transacao', 'id_entidade'])
    X = df[features_para_modelo]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    df_scaled = pd.DataFrame(X_scaled, columns=features_para_modelo, index=df.index)
    df_scaled[['complexidade', 'id_transacao']] = df[['complexidade', 'id_transacao']]

    # --- Treino e Predição ---
    print("--- Treinando e prevendo com os modelos... ---")
    df['anomalia_detectada'] = 'Normal'
    df['modelo_deteccao'] = 'N/A'

    # Isolation Forest
    indices_simples = df_scaled[df_scaled['complexidade'] == 'Simples'].index
    df_simples = df_scaled.loc[indices_simples].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_simples.empty:
        iso_forest = IsolationForest(n_estimators=200, contamination=0.01, random_state=42, n_jobs=-1)
        iso_forest.fit(df_simples)
        predicoes_if = iso_forest.predict(df_simples)
        df.loc[indices_simples, 'anomalia_detectada'] = np.where(predicoes_if == -1, 'Anómala', 'Normal')
        df.loc[indices_simples, 'modelo_deteccao'] = 'Isolation Forest'

    # Autoencoder
    indices_complexas = df_scaled[df_scaled['complexidade'] == 'Complexa'].index
    df_complexas = df_scaled.loc[indices_complexas].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_complexas.empty:
        input_dim = df_complexas.shape[1]
        input_layer = Input(shape=(input_dim,))
        encoder = Dense(16, activation='relu')(input_layer)
        encoder = Dense(8, activation='relu')(encoder)
        decoder = Dense(16, activation='relu')(encoder)
        decoder = Dense(input_dim, activation='linear')(decoder)
        autoencoder = Model(inputs=input_layer, outputs=decoder)
        autoencoder.compile(optimizer='adam', loss='mean_squared_error')
        autoencoder.fit(df_complexas, df_complexas, epochs=50, batch_size=32, verbose=0)
        reconstrucoes = autoencoder.predict(df_complexas)
        mse = np.mean(np.power(df_complexas.values - reconstrucoes, 2), axis=1)
        threshold = np.quantile(mse, 0.95)
        predicoes_ae = np.where(mse > threshold, -1, 1)
        df.loc[indices_complexas, 'anomalia_detectada'] = np.where(predicoes_ae == -1, 'Anómala', 'Normal')
        df.loc[indices_complexas, 'modelo_deteccao'] = 'Autoencoder'

    # Renomear 'categoria' para 'conta' para corresponder à interface do dashboard
    df.rename(columns={'categoria': 'conta'}, inplace=True)

    print("Preparação dos dados concluída.")
    return df.sort_index()

# ==============================================================================
# 3. EXECUÇÃO DO PIPELINE E PREPARAÇÃO DOS DADOS FINAIS
# ==============================================================================
df_final = preparar_dados_para_dashboard()

# ==============================================================================
# 4. INICIALIZAÇÃO E LAYOUT DA APLICAÇÃO DASH (VERSÃO CORRIGIDA PARA COLAB)
# ==============================================================================
# CORREÇÃO: Usar Dash e especificar o modo para jupyter/colab na inicialização
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], jupyter_mode="inline")
app.title = "Dashboard de Auditoria Financeira"

app.layout = dbc.Container([
    html.H1("Dashboard de Auditoria de Anomalias Financeiras", className="my-4 text-center"),
    html.H2("Caso de Estudo: SBD (com Pipeline Otimizado)", className="text-center text-muted mb-4"),

    dbc.Row([
        dbc.Col([
            html.Label("Conta:"),
            dcc.Dropdown(
                id="conta-dropdown",
                options=[{"label": c, "value": c} for c in ["Todos"] + sorted(df_final["conta"].unique())],
                value="Todos"
            )
        ], md=4),

        dbc.Col([
            html.Label("Tipo de Complexidade:"),
            dcc.Dropdown(
                id="tipo-dropdown",
                options=[{"label": c, "value": c} for c in ["Todos"] + df_final["complexidade"].unique().tolist()],
                value="Todos"
            )
        ], md=4),

        dbc.Col([
            html.Label("Estado da Anomalia:"),
            dcc.Dropdown(
                id="anomalia-dropdown",
                options=[
                    {"label": "Todas", "value": "Todas"},
                    {"label": "Apenas Anômalas", "value": "Anómalas"},
                    {"label": "Apenas Normais", "value": "Normais"}
                ],
                value="Todas"
            )
        ], md=4)
    ], className="mb-4"),

    dcc.Graph(id="scatter-graph"),

    dash_table.DataTable(
        id="table",
        columns=[{"name": col, "id": col} for col in ["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]],
        page_size=10,
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left"},
        style_header={'backgroundColor': '#003366', 'color': 'white', 'fontWeight': 'bold'},
        sort_action="native",
        filter_action="native",
    )
], fluid=True)

# ==============================================================================
# 5. CALLBACK - LÓGICA INTERATIVA DO DASHBOARD
# ==============================================================================
@app.callback(
    [Output("scatter-graph", "figure"),
     Output("table", "data")],
    [DashInput("conta-dropdown", "value"),
     DashInput("tipo-dropdown", "value"),
     DashInput("anomalia-dropdown", "value")]
)
def actualizar_dashboard(conta, tipo, anomalia):
    df_filtrado = df_final.copy()

    if conta != "Todos":
        df_filtrado = df_filtrado[df_filtrado["conta"] == conta]
    if tipo != "Todos":
        df_filtrado = df_filtrado[df_filtrado["complexidade"] == tipo]

    if anomalia == "Anómalas":
        df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Anómala"]
    elif anomalia == "Normais":
        df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Normal"]

    fig = px.scatter(
        df_filtrado, x="valor", y="dia_do_ano",
        color="anomalia_detectada",
        hover_data=['conta', 'complexidade', 'modelo_deteccao'],
        color_discrete_map={'Anómala': '#FF4136', 'Normal': '#0074D9'},
        title="Visualização de Anomalias (Valor vs. Dia do Ano)",
        labels={"valor": "Valor da Transação (MZN)", "dia_do_ano": "Dia do Ano"}
    )
    fig.update_layout(transition_duration=500)

    data_tabela = df_filtrado[["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]].to_dict("records")

    return fig, data_tabela

# ==============================================================================
# 6. EXECUÇÃO DA APLICAÇÃO NO COLAB (FORMA CORRETA)
# ==============================================================================
if __name__ == "__main__":
    # CORREÇÃO: Usar app.run() para a nova versão do Dash
    app.run(debug=True)

Bibliotecas importadas com sucesso.

--- A iniciar a preparação completa dos dados... ---
--- A simular dados de exemplo... ---
--- Engenharia de Atributos... ---
--- Definindo a Complexidade... ---
--- Treinando e prevendo com os modelos... ---
38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Preparação dos dados concluída.


TypeError: Dash() got an unexpected keyword argument 'jupyter_mode'

In [ ]:
# ==============================================================================
# 0. SETUP PARA GOOGLE COLAB - INSTALAÇÃO E ATUALIZAÇÃO DE BIBLIOTECAS
# ==============================================================================
# CORREÇÃO: Usar --upgrade para garantir que temos a versão mais recente do Dash (v2.0+)
!pip install --upgrade dash dash-bootstrap-components plotly pandas scikit-learn tensorflow -q

# ==============================================================================
# 1. SETUP - IMPORTAÇÃO DAS BIBLIOTECAS
# ==============================================================================
from dash import Dash # Usar a biblioteca Dash principal
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
from dash.dependencies import Input as DashInput, Output
import plotly.express as px
import pandas as pd
import numpy as np

# Bibliotecas de Machine Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

print("Bibliotecas importadas com sucesso.")

# ==============================================================================
# 2. FUNÇÃO DO PIPELINE DE DADOS E MODELAGEM
# ==============================================================================
def preparar_dados_para_dashboard():
    """
    Executa o pipeline completo: simulação, engenharia de atributos, modelagem
    e predição. Retorna um DataFrame final pronto para o dashboard.
    """
    print("\n--- A iniciar a preparação completa dos dados... ---")

    # --- Simulação de Dados (SUBSTITUA PELOS SEUS DADOS REAIS) ---
    print("--- A simular dados de exemplo... ---")
    num_registos = 10000
    data = {
        'id_transacao': range(num_registos),
        'data_hora': pd.to_datetime(np.random.choice(pd.date_range('2024-01-01', '2025-01-01', freq='h'), num_registos)),
        'valor': np.random.lognormal(mean=8, sigma=1.5, size=num_registos).round(2),
        'tipo_transacao': np.random.choice(['Pagamento', 'Recebimento', 'Transferência'], size=num_registos),
        'categoria': np.random.choice(['Conta Corrente', 'Salários', 'Marketing', 'Diversos', 'Impostos'], size=num_registos, p=[0.4, 0.2, 0.2, 0.1, 0.1]),
        'id_entidade': np.random.randint(1, 50, size=num_registos)
    }
    df = pd.DataFrame(data)
    df.loc[df.sample(frac=0.02).index, 'valor'] *= np.random.uniform(5, 10)
    df.loc[df.sample(frac=0.01).index, 'valor'] *= -1
    df.loc[df.sample(frac=0.01).index, 'data_hora'] += pd.to_timedelta(np.random.randint(21, 23), unit='h')

    # --- Engenharia de Atributos ---
    print("--- Engenharia de Atributos... ---")
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    df['dia_do_ano'] = df['data_hora'].dt.dayofyear
    mediana_por_entidade = df.groupby('id_entidade')['valor'].transform('median')
    df['desvio_mediana_historica'] = (df['valor'] - mediana_por_entidade) / (mediana_por_entidade + 1e-6)
    df['hora_incomum'] = ((df['data_hora'].dt.hour < 7) | (df['data_hora'].dt.hour > 20)).astype(int)
    df['dia_fim_de_semana'] = (df['data_hora'].dt.dayofweek >= 5).astype(int)
    df['arredondamento_valor'] = (df['valor'] > 1000) & (df['valor'] % 1000 == 0).astype(int)
    df = df.sort_values(by=['id_entidade', 'data_hora'])
    diferenca_tempo = df.groupby('id_entidade')['data_hora'].diff().dt.total_seconds()
    df['sequencia_rapida'] = (diferenca_tempo < 60).astype(int)
    df.fillna(0, inplace=True)

    # --- Definição da Complexidade ---
    print("--- Definindo a Complexidade... ---")
    media_geral = df['valor'].mean()
    std_geral = df['valor'].std()
    criterio_1 = df['valor'] > (media_geral + 3 * std_geral)
    criterio_2 = df['categoria'].isin(['Diversos', 'Outros'])
    criterio_3 = df['valor'] < 0
    df['complexidade'] = np.where((criterio_1 | criterio_2 | criterio_3), 'Complexa', 'Simples')

    # --- Pré-processamento e Divisão ---
    features_para_modelo = df.select_dtypes(include=np.number).columns.drop(['id_transacao', 'id_entidade'])
    X = df[features_para_modelo]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    df_scaled = pd.DataFrame(X_scaled, columns=features_para_modelo, index=df.index)
    df_scaled[['complexidade', 'id_transacao']] = df[['complexidade', 'id_transacao']]

    # --- Treino e Predição ---
    print("--- Treinando e prevendo com os modelos... ---")
    df['anomalia_detectada'] = 'Normal'
    df['modelo_deteccao'] = 'N/A'

    # Isolation Forest
    indices_simples = df_scaled[df_scaled['complexidade'] == 'Simples'].index
    df_simples = df_scaled.loc[indices_simples].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_simples.empty:
        iso_forest = IsolationForest(n_estimators=200, contamination=0.01, random_state=42, n_jobs=-1)
        iso_forest.fit(df_simples)
        predicoes_if = iso_forest.predict(df_simples)
        df.loc[indices_simples, 'anomalia_detectada'] = np.where(predicoes_if == -1, 'Anómala', 'Normal')
        df.loc[indices_simples, 'modelo_deteccao'] = 'Isolation Forest'

    # Autoencoder
    indices_complexas = df_scaled[df_scaled['complexidade'] == 'Complexa'].index
    df_complexas = df_scaled.loc[indices_complexas].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_complexas.empty:
        input_dim = df_complexas.shape[1]
        input_layer = Input(shape=(input_dim,))
        encoder = Dense(16, activation='relu')(input_layer)
        encoder = Dense(8, activation='relu')(encoder)
        decoder = Dense(16, activation='relu')(encoder)
        decoder = Dense(input_dim, activation='linear')(decoder)
        autoencoder = Model(inputs=input_layer, outputs=decoder)
        autoencoder.compile(optimizer='adam', loss='mean_squared_error')
        autoencoder.fit(df_complexas, df_complexas, epochs=50, batch_size=32, verbose=0)
        reconstrucoes = autoencoder.predict(df_complexas)
        mse = np.mean(np.power(df_complexas.values - reconstrucoes, 2), axis=1)
        threshold = np.quantile(mse, 0.95)
        predicoes_ae = np.where(mse > threshold, -1, 1)
        df.loc[indices_complexas, 'anomalia_detectada'] = np.where(predicoes_ae == -1, 'Anómala', 'Normal')
        df.loc[indices_complexas, 'modelo_deteccao'] = 'Autoencoder'

    # Renomear 'categoria' para 'conta' para corresponder à interface do dashboard
    df.rename(columns={'categoria': 'conta'}, inplace=True)

    print("Preparação dos dados concluída.")
    return df.sort_index()

# ==============================================================================
# 3. EXECUÇÃO DO PIPELINE E PREPARAÇÃO DOS DADOS FINAIS
# ==============================================================================
df_final = preparar_dados_para_dashboard()

# ==============================================================================
# 4. INICIALIZAÇÃO E LAYOUT DA APLICAÇÃO DASH
# ==============================================================================
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], jupyter_mode="inline")
app.title = "Dashboard de Auditoria Financeira"

app.layout = dbc.Container([
    html.H1("Dashboard de Auditoria de Anomalias Financeiras", className="my-4 text-center"),
    html.H2("Caso de Estudo: SBD (com Pipeline Otimizado)", className="text-center text-muted mb-4"),

    dbc.Row([
        dbc.Col([
            html.Label("Conta:"),
            dcc.Dropdown(
                id="conta-dropdown",
                options=[{"label": c, "value": c} for c in ["Todos"] + sorted(df_final["conta"].unique())],
                value="Todos"
            )
        ], md=4),

        dbc.Col([
            html.Label("Tipo de Complexidade:"),
            dcc.Dropdown(
                id="tipo-dropdown",
                options=[{"label": c, "value": c} for c in ["Todos"] + df_final["complexidade"].unique().tolist()],
                value="Todos"
            )
        ], md=4),

        dbc.Col([
            html.Label("Estado da Anomalia:"),
            dcc.Dropdown(
                id="anomalia-dropdown",
                options=[
                    {"label": "Todas", "value": "Todas"},
                    {"label": "Apenas Anômalas", "value": "Anómalas"},
                    {"label": "Apenas Normais", "value": "Normais"}
                ],
                value="Todas"
            )
        ], md=4)
    ], className="mb-4"),

    dcc.Graph(id="scatter-graph"),

    dash_table.DataTable(
        id="table",
        columns=[{"name": col, "id": col} for col in ["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]],
        page_size=10,
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left"},
        style_header={'backgroundColor': '#003366', 'color': 'white', 'fontWeight': 'bold'},
        sort_action="native",
        filter_action="native",
    )
], fluid=True)

# ==============================================================================
# 5. CALLBACK - LÓGICA INTERATIVA DO DASHBOARD
# ==============================================================================
@app.callback(
    [Output("scatter-graph", "figure"),
     Output("table", "data")],
    [DashInput("conta-dropdown", "value"),
     DashInput("tipo-dropdown", "value"),
     DashInput("anomalia-dropdown", "value")]
)
def actualizar_dashboard(conta, tipo, anomalia):
    df_filtrado = df_final.copy()

    if conta != "Todos":
        df_filtrado = df_filtrado[df_filtrado["conta"] == conta]
    if tipo != "Todos":
        df_filtrado = df_filtrado[df_filtrado["complexidade"] == tipo]

    if anomalia == "Anómalas":
        df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Anómala"]
    elif anomalia == "Normais":
        df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Normal"]

    fig = px.scatter(
        df_filtrado, x="valor", y="dia_do_ano",
        color="anomalia_detectada",
        hover_data=['conta', 'complexidade', 'modelo_deteccao'],
        color_discrete_map={'Anómala': '#FF4136', 'Normal': '#0074D9'},
        title="Visualização de Anomalias (Valor vs. Dia do Ano)",
        labels={"valor": "Valor da Transação (MZN)", "dia_do_ano": "Dia do Ano"}
    )
    fig.update_layout(transition_duration=500)

    data_tabela = df_filtrado[["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]].to_dict("records")

    return fig, data_tabela

# ==============================================================================
# 6. EXECUÇÃO DA APLICAÇÃO NO COLAB
# ==============================================================================
if __name__ == "__main__":
    app.run(debug=True)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 104.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.1 which is incompatible.
dask-cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
cudf-cu12 25.6.0 requires pandas<2.2.4dev0,>=2.0, but you have pandas 2.3.1 which is incompatible.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.1 which is incompatible.
Bibliotecas importadas com sucesso.

--- A iniciar a preparação completa dos dados... ---
--- A simular d

TypeError: Dash() got an unexpected keyword argument 'jupyter_mode'

In [ ]:
# ==============================================================================
# 0. SETUP - INSTALAÇÃO CUIDADOSA DE BIBLIOTECAS PARA O COLAB
# ==============================================================================
# CORREÇÃO: Pedimos uma versão do Dash >= 2.0 (para ter o jupyter_mode)
# mas sem o --upgrade, para tentar respeitar as dependências do Colab.
!pip install "dash>=2.0.0" dash-bootstrap-components plotly -q

# ==============================================================================
# 1. SETUP - IMPORTAÇÃO DAS BIBLIOTECAS
# ==============================================================================
from dash import Dash
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
from dash.dependencies import Input as DashInput, Output
import plotly.express as px
import pandas as pd
import numpy as np

# Bibliotecas de Machine Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

print("Bibliotecas importadas com sucesso.")

# ==============================================================================
# 2. FUNÇÃO DO PIPELINE DE DADOS E MODELAGEM
# ==============================================================================
def preparar_dados_para_dashboard():
    """
    Executa o pipeline completo: simulação, engenharia de atributos, modelagem
    e predição. Retorna um DataFrame final pronto para o dashboard.
    """
    print("\n--- A iniciar a preparação completa dos dados... ---")

    # --- Simulação de Dados (SUBSTITUA PELOS SEUS DADOS REAIS) ---
    print("--- A simular dados de exemplo... ---")
    num_registos = 10000
    data = {
        'id_transacao': range(num_registos),
        'data_hora': pd.to_datetime(np.random.choice(pd.date_range('2024-01-01', '2025-01-01', freq='h'), num_registos)),
        'valor': np.random.lognormal(mean=8, sigma=1.5, size=num_registos).round(2),
        'tipo_transacao': np.random.choice(['Pagamento', 'Recebimento', 'Transferência'], size=num_registos),
        'categoria': np.random.choice(['Conta Corrente', 'Salários', 'Marketing', 'Diversos', 'Impostos'], size=num_registos, p=[0.4, 0.2, 0.2, 0.1, 0.1]),
        'id_entidade': np.random.randint(1, 50, size=num_registos)
    }
    df = pd.DataFrame(data)
    df.loc[df.sample(frac=0.02).index, 'valor'] *= np.random.uniform(5, 10)
    df.loc[df.sample(frac=0.01).index, 'valor'] *= -1
    df.loc[df.sample(frac=0.01).index, 'data_hora'] += pd.to_timedelta(np.random.randint(21, 23), unit='h')

    # --- Engenharia de Atributos ---
    print("--- Engenharia de Atributos... ---")
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    df['dia_do_ano'] = df['data_hora'].dt.dayofyear
    mediana_por_entidade = df.groupby('id_entidade')['valor'].transform('median')
    df['desvio_mediana_historica'] = (df['valor'] - mediana_por_entidade) / (mediana_por_entidade + 1e-6)
    df['hora_incomum'] = ((df['data_hora'].dt.hour < 7) | (df['data_hora'].dt.hour > 20)).astype(int)
    df['dia_fim_de_semana'] = (df['data_hora'].dt.dayofweek >= 5).astype(int)
    df['arredondamento_valor'] = (df['valor'] > 1000) & (df['valor'] % 1000 == 0).astype(int)
    df = df.sort_values(by=['id_entidade', 'data_hora'])
    diferenca_tempo = df.groupby('id_entidade')['data_hora'].diff().dt.total_seconds()
    df['sequencia_rapida'] = (diferenca_tempo < 60).astype(int)
    df.fillna(0, inplace=True)

    # --- Definição da Complexidade ---
    print("--- Definindo a Complexidade... ---")
    media_geral = df['valor'].mean()
    std_geral = df['valor'].std()
    criterio_1 = df['valor'] > (media_geral + 3 * std_geral)
    criterio_2 = df['categoria'].isin(['Diversos', 'Outros'])
    criterio_3 = df['valor'] < 0
    df['complexidade'] = np.where((criterio_1 | criterio_2 | criterio_3), 'Complexa', 'Simples')

    # --- Pré-processamento e Divisão ---
    features_para_modelo = df.select_dtypes(include=np.number).columns.drop(['id_transacao', 'id_entidade'])
    X = df[features_para_modelo]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    df_scaled = pd.DataFrame(X_scaled, columns=features_para_modelo, index=df.index)
    df_scaled[['complexidade', 'id_transacao']] = df[['complexidade', 'id_transacao']]

    # --- Treino e Predição ---
    print("--- Treinando e prevendo com os modelos... ---")
    df['anomalia_detectada'] = 'Normal'
    df['modelo_deteccao'] = 'N/A'

    # Isolation Forest
    indices_simples = df_scaled[df_scaled['complexidade'] == 'Simples'].index
    df_simples = df_scaled.loc[indices_simples].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_simples.empty:
        iso_forest = IsolationForest(n_estimators=200, contamination=0.01, random_state=42, n_jobs=-1)
        iso_forest.fit(df_simples)
        predicoes_if = iso_forest.predict(df_simples)
        df.loc[indices_simples, 'anomalia_detectada'] = np.where(predicoes_if == -1, 'Anómala', 'Normal')
        df.loc[indices_simples, 'modelo_deteccao'] = 'Isolation Forest'

    # Autoencoder
    indices_complexas = df_scaled[df_scaled['complexidade'] == 'Complexa'].index
    df_complexas = df_scaled.loc[indices_complexas].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_complexas.empty:
        input_dim = df_complexas.shape[1]
        input_layer = Input(shape=(input_dim,))
        encoder = Dense(16, activation='relu')(input_layer)
        encoder = Dense(8, activation='relu')(encoder)
        decoder = Dense(16, activation='relu')(encoder)
        decoder = Dense(input_dim, activation='linear')(decoder)
        autoencoder = Model(inputs=input_layer, outputs=decoder)
        autoencoder.compile(optimizer='adam', loss='mean_squared_error')
        autoencoder.fit(df_complexas, df_complexas, epochs=50, batch_size=32, verbose=0)
        reconstrucoes = autoencoder.predict(df_complexas)
        mse = np.mean(np.power(df_complexas.values - reconstrucoes, 2), axis=1)
        threshold = np.quantile(mse, 0.95)
        predicoes_ae = np.where(mse > threshold, -1, 1)
        df.loc[indices_complexas, 'anomalia_detectada'] = np.where(predicoes_ae == -1, 'Anómala', 'Normal')
        df.loc[indices_complexas, 'modelo_deteccao'] = 'Autoencoder'

    # Renomear 'categoria' para 'conta' para corresponder à interface do dashboard
    df.rename(columns={'categoria': 'conta'}, inplace=True)

    print("Preparação dos dados concluída.")
    return df.sort_index()

# ==============================================================================
# 3. EXECUÇÃO DO PIPELINE E PREPARAÇÃO DOS DADOS FINAIS
# ==============================================================================
df_final = preparar_dados_para_dashboard()

# ==============================================================================
# 4. INICIALIZAÇÃO E LAYOUT DA APLICAÇÃO DASH
# ==============================================================================
# CORREÇÃO: Removido jupyter_mode="inline" daqui
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Dashboard de Auditoria Financeira"

app.layout = dbc.Container([
    html.H1("Dashboard de Auditoria de Anomalias Financeiras", className="my-4 text-center"),
    html.H2("Caso de Estudo: SBD Limitada", className="text-center text-muted mb-4"),

    dbc.Row([
        dbc.Col([
            html.Label("Conta:"),
            dcc.Dropdown(
                id="conta-dropdown",
                options=[{"label": c, "value": c} for c in ["Todos"] + sorted(df_final["conta"].unique())],
                value="Todos"
            )
        ], md=4),

        dbc.Col([
            html.Label("Tipo de Complexidade:"),
            dcc.Dropdown(
                id="tipo-dropdown",
                options=[{"label": c, "value": c} for c in ["Todos"] + df_final["complexidade"].unique().tolist()],
                value="Todos"
            )
        ], md=4),

        dbc.Col([
            html.Label("Estado da Anomalia:"),
            dcc.Dropdown(
                id="anomalia-dropdown",
                options=[
                    {"label": "Todas", "value": "Todas"},
                    {"label": "Apenas Anômalas", "value": "Anómalas"},
                    {"label": "Apenas Normais", "value": "Normais"}
                ],
                value="Todas"
            )
        ], md=4)
    ], className="mb-4"),

    dcc.Graph(id="scatter-graph"),

    dash_table.DataTable(
        id="table",
        columns=[{"name": col, "id": col} for col in ["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]],
        page_size=10,
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left"},
        style_header={'backgroundColor': '#003366', 'color': 'white', 'fontWeight': 'bold'},
        sort_action="native",
        filter_action="native",
    )
], fluid=True)

# ==============================================================================
# 5. CALLBACK - LÓGICA INTERATIVA DO DASHBOARD
# ==============================================================================
@app.callback(
    [Output("scatter-graph", "figure"),
     Output("table", "data")],
    [DashInput("conta-dropdown", "value"),
     DashInput("tipo-dropdown", "value"),
     DashInput("anomalia-dropdown", "value")]
)
def actualizar_dashboard(conta, tipo, anomalia):
    df_filtrado = df_final.copy()

    if conta != "Todos":
        df_filtrado = df_filtrado[df_filtrado["conta"] == conta]
    if tipo != "Todos":
        df_filtrado = df_filtrado[df_filtrado["complexidade"] == tipo]

    if anomalia == "Anómalas":
        df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Anómala"]
    elif anomalia == "Normais":
        df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Normal"]

    fig = px.scatter(
        df_filtrado, x="valor", y="dia_do_ano",
        color="anomalia_detectada",
        hover_data=['conta', 'complexidade', 'modelo_deteccao'],
        color_discrete_map={'Anómala': '#FF4136', 'Normal': '#0074D9'},
        title="Visualização de Anomalias (Valor vs. Dia do Ano)",
        labels={"valor": "Valor da Transação (MZN)", "dia_do_ano": "Dia do Ano"}
    )
    fig.update_layout(transition_duration=500)

    data_tabela = df_filtrado[["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]].to_dict("records")

    return fig, data_tabela

# ==============================================================================
# 6. EXECUÇÃO DA APLICAÇÃO NO COLAB
# ==============================================================================
if __name__ == "__main__":
    # CORREÇÃO: Usar app.run_server com mode='inline'
    app.run(mode='inline') # Use mode='inline' for Colab

Bibliotecas importadas com sucesso.

--- A iniciar a preparação completa dos dados... ---
--- A simular dados de exemplo... ---
--- Engenharia de Atributos... ---
--- Definindo a Complexidade... ---
--- Treinando e prevendo com os modelos... ---
39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Preparação dos dados concluída.


<IPython.core.display.Javascript object>

In [ ]:
# ==============================================================================
# 0. SETUP - INSTALAÇÃO DE BIBLIOTECAS
# ==============================================================================
!pip install "dash==2.10.2" "jupyter-dash==0.4.2" dash-bootstrap-components plotly openpyxl -q

# ==============================================================================
# 1. SETUP - IMPORTAÇÃO DAS BIBLIOTECAS
# ==============================================================================
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
from dash.dependencies import Input as DashInput, Output, State
import plotly.express as px
import pandas as pd
import numpy as np
import base64
import io

# Bibliotecas de Machine Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

print("Bibliotecas importadas com sucesso.")

# ==============================================================================
# 2. FUNÇÃO DO PIPELINE DE DADOS E MODELAGEM (MODIFICADA)
# ==============================================================================
# MODIFICAÇÃO: A função agora recebe um DataFrame como argumento.
def executar_pipeline_de_analise(df_inicial):
    """
    Executa o pipeline completo sobre um DataFrame fornecido.
    Retorna um DataFrame final com os resultados da análise.
    """
    print("\n--- A executar o pipeline de análise sobre os novos dados... ---")
    df = df_inicial.copy()

    # --- Engenharia de Atributos ---
    print("--- Engenharia de Atributos... ---")
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    df['dia_do_ano'] = df['data_hora'].dt.dayofyear
    mediana_por_entidade = df.groupby('id_entidade')['valor'].transform('median')
    df['desvio_mediana_historica'] = (df['valor'] - mediana_por_entidade) / (mediana_por_entidade + 1e-6)
    df['hora_incomum'] = ((df['data_hora'].dt.hour < 7) | (df['data_hora'].dt.hour > 20)).astype(int)
    df['dia_fim_de_semana'] = (df['data_hora'].dt.dayofweek >= 5).astype(int)
    df['arredondamento_valor'] = (df['valor'] > 1000) & (df['valor'] % 1000 == 0).astype(int)
    df = df.sort_values(by=['id_entidade', 'data_hora'])
    diferenca_tempo = df.groupby('id_entidade')['data_hora'].diff().dt.total_seconds()
    df['sequencia_rapida'] = (diferenca_tempo < 60).astype(int)
    df.fillna(0, inplace=True)

    # --- Definição da Complexidade ---
    print("--- Definindo a Complexidade... ---")
    media_geral = df['valor'].mean()
    std_geral = df['valor'].std()
    criterio_1 = df['valor'] > (media_geral + 3 * std_geral)
    # MODIFICAÇÃO: Usamos 'conta' que pode ser o nome da coluna no ficheiro do utilizador
    criterio_2 = df['conta'].isin(['Diversos', 'Outros'])
    criterio_3 = df['valor'] < 0
    df['complexidade'] = np.where((criterio_1 | criterio_2 | criterio_3), 'Complexa', 'Simples')

    # --- Pré-processamento e Divisão ---
    features_para_modelo = df.select_dtypes(include=np.number).columns.drop(['id_transacao', 'id_entidade'], errors='ignore')
    X = df[features_para_modelo]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    df_scaled = pd.DataFrame(X_scaled, columns=features_para_modelo, index=df.index)
    df_scaled[['complexidade', 'id_transacao']] = df[['complexidade', 'id_transacao']]

    # --- Treino e Predição ---
    print("--- Treinando e prevendo com os modelos... ---")
    df['anomalia_detectada'] = 'Normal'
    df['modelo_deteccao'] = 'N/A'

    # Isolation Forest
    indices_simples = df_scaled[df_scaled['complexidade'] == 'Simples'].index
    df_simples = df_scaled.loc[indices_simples].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_simples.empty:
        iso_forest = IsolationForest(n_estimators=200, contamination=0.01, random_state=42, n_jobs=-1)
        iso_forest.fit(df_simples)
        predicoes_if = iso_forest.predict(df_simples)
        df.loc[indices_simples, 'anomalia_detectada'] = np.where(predicoes_if == -1, 'Anómala', 'Normal')
        df.loc[indices_simples, 'modelo_deteccao'] = 'Isolation Forest'

    # Autoencoder
    indices_complexas = df_scaled[df_scaled['complexidade'] == 'Complexa'].index
    df_complexas = df_scaled.loc[indices_complexas].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_complexas.empty:
        input_dim = df_complexas.shape[1]
        input_layer = Input(shape=(input_dim,))
        encoder = Dense(16, activation='relu')(input_layer)
        encoder = Dense(8, activation='relu')(encoder)
        decoder = Dense(16, activation='relu')(encoder)
        decoder = Dense(input_dim, activation='linear')(decoder)
        autoencoder = Model(inputs=input_layer, outputs=decoder)
        autoencoder.compile(optimizer='adam', loss='mean_squared_error')
        autoencoder.fit(df_complexas, df_complexas, epochs=50, batch_size=32, verbose=0)
        reconstrucoes = autoencoder.predict(df_complexas)
        mse = np.mean(np.power(df_complexas.values - reconstrucoes, 2), axis=1)
        threshold = np.quantile(mse, 0.95)
        predicoes_ae = np.where(mse > threshold, -1, 1)
        df.loc[indices_complexas, 'anomalia_detectada'] = np.where(predicoes_ae == -1, 'Anómala', 'Normal')
        df.loc[indices_complexas, 'modelo_deteccao'] = 'Autoencoder'

    print("Pipeline de análise concluído.")
    return df.sort_index()

# ==============================================================================
# 3. INICIALIZAÇÃO E LAYOUT DA APLICAÇÃO DASH (COM UPLOAD)
# ==============================================================================
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Dashboard de Auditoria Financeira"

app.layout = dbc.Container([
    # MODIFICAÇÃO: Componente para armazenar os dados processados em formato JSON
    dcc.Store(id='dados-processados-memoria'),

    html.H1("Dashboard de Auditoria de Anomalias Financeiras", className="my-4 text-center"),

    # MODIFICAÇÃO: Secção de Upload de Ficheiro
    dbc.Row([
        dbc.Col([
            dcc.Upload(
                id='upload-dados',
                children=html.Div([
                    'Arraste e Solte ou ',
                    html.A('Selecione um Ficheiro')
                ]),
                style={
                    'width': '100%', 'height': '60px', 'lineHeight': '60px',
                    'borderWidth': '1px', 'borderStyle': 'dashed',
                    'borderRadius': '5px', 'textAlign': 'center', 'margin': '10px'
                },
                multiple=False # Apenas um ficheiro de cada vez
            ),
            html.Div(id='output-upload-state') # Para mostrar o estado do upload
        ])
    ], className="mb-4"),

    html.Hr(),

    # Painel de Filtros (igual ao anterior)
    dbc.Row([
        dbc.Col([
            html.Label("Conta:"),
            dcc.Dropdown(id="conta-dropdown", value="Todos")
        ], md=4),
        dbc.Col([
            html.Label("Tipo de Complexidade:"),
            dcc.Dropdown(id="tipo-dropdown", value="Todos")
        ], md=4),
        dbc.Col([
            html.Label("Estado da Anomalia:"),
            dcc.Dropdown(id="anomalia-dropdown", value="Todas")
        ], md=4)
    ], className="mb-4"),

    dcc.Graph(id="scatter-graph"),
    dash_table.DataTable(id="table", page_size=10, style_table={"overflowX": "auto"},
                         style_cell={"textAlign": "left"}, style_header={'backgroundColor': '#003366', 'color': 'white', 'fontWeight': 'bold'},
                         sort_action="native", filter_action="native")
], fluid=True)

# ==============================================================================
# 4. CALLBACKS - LÓGICA INTERATIVA DO DASHBOARD
# ==============================================================================

# MODIFICAÇÃO: Novo callback para processar o ficheiro carregado
@app.callback(
    Output('dados-processados-memoria', 'data'),
    Output('output-upload-state', 'children'),
    Input('upload-dados', 'contents'),
    State('upload-dados', 'filename')
)
def processar_ficheiro_carregado(contents, filename):
    if contents is None:
        return None, "Por favor, carregue um ficheiro (CSV ou Excel) para iniciar a análise."

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)

    try:
        if 'csv' in filename:
            # Assume que o utilizador carrega um ficheiro CSV
            df_inicial = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename or 'xlsx' in filename:
            # Assume que o utilizador carrega um ficheiro Excel
            df_inicial = pd.read_excel(io.BytesIO(decoded))
        else:
            return None, html.Div(['Tipo de ficheiro não suportado. Por favor, use CSV ou Excel.'], style={'color': 'red'})

        # MODIFICAÇÃO: Renomear 'categoria' para 'conta' logo no início
        if 'categoria' in df_inicial.columns:
            df_inicial.rename(columns={'categoria': 'conta'}, inplace=True)

        # Executar todo o pipeline de ML
        df_processado = executar_pipeline_de_analise(df_inicial)

        # Converter para JSON para armazenar no dcc.Store
        json_data = df_processado.to_json(date_format='iso', orient='split')

        return json_data, html.Div([f'Ficheiro "{filename}" carregado e processado com sucesso!'], style={'color': 'green'})

    except Exception as e:
        print(e)
        return None, html.Div(['Ocorreu um erro ao processar o ficheiro.'], style={'color': 'red'})

# MODIFICAÇÃO: O callback principal agora depende dos dados armazenados e atualiza os filtros
@app.callback(
    Output("scatter-graph", "figure"),
    Output("table", "data"),
    Output("table", "columns"),
    Output("conta-dropdown", "options"),
    Output("tipo-dropdown", "options"),
    Output("anomalia-dropdown", "options"),
    Input('dados-processados-memoria', 'data'),
    Input("conta-dropdown", "value"),
    Input("tipo-dropdown", "value"),
    Input("anomalia-dropdown", "value")
)
def actualizar_dashboard(json_data, conta, tipo, anomalia):
    if json_data is None:
        # Estado inicial antes de carregar um ficheiro
        return {}, [], [], [], [], []

    df_final = pd.read_json(json_data, orient='split')

    # Atualizar opções dos filtros com base nos dados carregados
    opcoes_conta = [{"label": c, "value": c} for c in ["Todos"] + sorted(df_final["conta"].unique())]
    opcoes_tipo = [{"label": c, "value": c} for c in ["Todos"] + df_final["complexidade"].unique().tolist()]
    opcoes_anomalia = [
        {"label": "Todas", "value": "Todas"},
        {"label": "Apenas Anômalas", "value": "Anómalas"},
        {"label": "Apenas Normais", "value": "Normais"}
    ]

    # Lógica de filtragem
    df_filtrado = df_final.copy()
    if conta and conta != "Todos":
        df_filtrado = df_filtrado[df_filtrado["conta"] == conta]
    if tipo and tipo != "Todos":
        df_filtrado = df_filtrado[df_filtrado["complexidade"] == tipo]
    if anomalia and anomalia == "Anómalas":
        df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Anómala"]
    elif anomalia and anomalia == "Normais":
        df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Normal"]

    # Criar o gráfico
    fig = px.scatter(
        df_filtrado, x="valor", y="dia_do_ano",
        color="anomalia_detectada",
        hover_data=['conta', 'complexidade', 'modelo_deteccao'],
        color_discrete_map={'Anómala': '#FF4136', 'Normal': '#0074D9'},
        title="Visualização de Anomalias (Valor vs. Dia do Ano)",
        labels={"valor": "Valor da Transação (MZN)", "dia_do_ano": "Dia do Ano"}
    )
    fig.update_layout(transition_duration=500)

    # Preparar dados para a tabela
    colunas_tabela = [{"name": col, "id": col} for col in ["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]]
    data_tabela = df_filtrado[colunas_tabela_ids].to_dict("records")

    return fig, data_tabela, colunas_tabela, opcoes_conta, opcoes_tipo, opcoes_anomalia

# ==============================================================================
# 5. EXECUÇÃO DA APLICAÇÃO NO COLAB
# ==============================================================================
if __name__ == "__main__":
    app.run_server(mode='inline')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 15.9 MB/s eta 0:00:00
Bibliotecas importadas com sucesso.


ValueError: Cannot convert '('u', 'p', 'l', 'o', 'a', 'd', '-', 'd', 'a', 'd', 'o', 's')' to a shape. Found invalid entry 'u' of type '<class 'str'>'. 

In [ ]:
# ==============================================================================
# 0. SETUP - INSTALAÇÃO DE VERSÕES ESPECÍFICAS E COMPATÍVEIS
# ==============================================================================
# Instalamos versões específicas que são conhecidas por funcionarem bem juntas.
!pip install "dash==2.10.2" "jupyter-dash==0.4.2" dash-bootstrap-components plotly openpyxl -q

# ==============================================================================
# 1. SETUP - IMPORTAÇÃO DAS BIBLIOTECAS (COM CORREÇÃO FINAL)
# ==============================================================================
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
# CORREÇÃO: Garantir que Input e State vêm do Dash, e usar um alias para Input.
from dash.dependencies import Input as DashInput, Output, State
import plotly.express as px
import pandas as pd
import numpy as np
import base64
import io

# Bibliotecas de Machine Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense # O Input do Keras mantém o nome original

print("Bibliotecas importadas com sucesso.")

# ==============================================================================
# 2. FUNÇÃO DO PIPELINE DE DADOS E MODELAGEM
# ==============================================================================
def executar_pipeline_de_analise(df_inicial):
    """
    Executa o pipeline completo sobre um DataFrame fornecido.
    Retorna um DataFrame final com os resultados da análise.
    """
    print("\n--- A executar o pipeline de análise sobre os novos dados... ---")
    df = df_inicial.copy()

    # --- Engenharia de Atributos ---
    print("--- Engenharia de Atributos... ---")
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    df['dia_do_ano'] = df['data_hora'].dt.dayofyear
    mediana_por_entidade = df.groupby('id_entidade')['valor'].transform('median')
    df['desvio_mediana_historica'] = (df['valor'] - mediana_por_entidade) / (mediana_por_entidade + 1e-6)
    df['hora_incomum'] = ((df['data_hora'].dt.hour < 7) | (df['data_hora'].dt.hour > 20)).astype(int)
    df['dia_fim_de_semana'] = (df['data_hora'].dt.dayofweek >= 5).astype(int)
    df['arredondamento_valor'] = (df['valor'] > 1000) & (df['valor'] % 1000 == 0).astype(int)
    df = df.sort_values(by=['id_entidade', 'data_hora'])
    diferenca_tempo = df.groupby('id_entidade')['data_hora'].diff().dt.total_seconds()
    df['sequencia_rapida'] = (diferenca_tempo < 60).astype(int)
    df.fillna(0, inplace=True)

    # --- Definição da Complexidade ---
    print("--- Definindo a Complexidade... ---")
    media_geral = df['valor'].mean()
    std_geral = df['valor'].std()
    criterio_1 = df['valor'] > (media_geral + 3 * std_geral)
    # Assumimos que a coluna 'conta' existe após o carregamento e renomeação
    criterio_2 = df['conta'].isin(['Diversos', 'Outros'])
    criterio_3 = df['valor'] < 0
    df['complexidade'] = np.where((criterio_1 | criterio_2 | criterio_3), 'Complexa', 'Simples')

    # --- Pré-processamento e Divisão ---
    features_para_modelo = df.select_dtypes(include=np.number).columns.drop(['id_transacao', 'id_entidade'], errors='ignore')
    X = df[features_para_modelo]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    df_scaled = pd.DataFrame(X_scaled, columns=features_para_modelo, index=df.index)
    df_scaled[['complexidade', 'id_transacao']] = df[['complexidade', 'id_transacao']]

    # --- Treino e Predição ---
    print("--- Treinando e prevendo com os modelos... ---")
    df['anomalia_detectada'] = 'Normal'
    df['modelo_deteccao'] = 'N/A'

    # Isolation Forest
    indices_simples = df_scaled[df_scaled['complexidade'] == 'Simples'].index
    df_simples = df_scaled.loc[indices_simples].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_simples.empty:
        iso_forest = IsolationForest(n_estimators=200, contamination=0.01, random_state=42, n_jobs=-1)
        iso_forest.fit(df_simples)
        predicoes_if = iso_forest.predict(df_simples)
        df.loc[indices_simples, 'anomalia_detectada'] = np.where(predicoes_if == -1, 'Anómala', 'Normal')
        df.loc[indices_simples, 'modelo_deteccao'] = 'Isolation Forest'

    # Autoencoder
    indices_complexas = df_scaled[df_scaled['complexidade'] == 'Complexa'].index
    df_complexas = df_scaled.loc[indices_complexas].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_complexas.empty:
        input_dim = df_complexas.shape[1]
        input_layer = Input(shape=(input_dim,)) # Este Input é do Keras
        encoder = Dense(16, activation='relu')(input_layer)
        encoder = Dense(8, activation='relu')(encoder)
        decoder = Dense(16, activation='relu')(encoder)
        decoder = Dense(input_dim, activation='linear')(decoder)
        autoencoder = Model(inputs=input_layer, outputs=decoder)
        autoencoder.compile(optimizer='adam', loss='mean_squared_error')
        autoencoder

Bibliotecas importadas com sucesso.


In [ ]:
# ==============================================================================
# 0. SETUP - INSTALAÇÃO DE VERSÕES ESPECÍFICAS E COMPATÍVEIS
# ==============================================================================
# Instalamos versões específicas que são conhecidas por funcionarem bem juntas.
!pip install "dash==2.10.2" "jupyter-dash==0.4.2" dash-bootstrap-components plotly openpyxl -q

# ==============================================================================
# 1. SETUP - IMPORTAÇÃO DAS BIBLIOTECAS (COM CORREÇÃO FINAL)
# ==============================================================================
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
# CORREÇÃO: Garantir que Input e State vêm do Dash, e usar um alias para Input.
from dash.dependencies import Input as DashInput, Output, State
import plotly.express as px
import pandas as pd
import numpy as np
import base64
import io

# Bibliotecas de Machine Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense # O Input do Keras mantém o nome original

print("Bibliotecas importadas com sucesso.")

# ==============================================================================
# 2. FUNÇÃO DO PIPELINE DE DADOS E MODELAGEM
# ==============================================================================
def executar_pipeline_de_analise(df_inicial):
    """
    Executa o pipeline completo sobre um DataFrame fornecido.
    Retorna um DataFrame final com os resultados da análise.
    """
    print("\n--- A executar o pipeline de análise sobre os novos dados... ---")
    df = df_inicial.copy()

    # --- Engenharia de Atributos ---
    print("--- Engenharia de Atributos... ---")
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    df['dia_do_ano'] = df['data_hora'].dt.dayofyear
    mediana_por_entidade = df.groupby('id_entidade')['valor'].transform('median')
    df['desvio_mediana_historica'] = (df['valor'] - mediana_por_entidade) / (mediana_por_entidade + 1e-6)
    df['hora_incomum'] = ((df['data_hora'].dt.hour < 7) | (df['data_hora'].dt.hour > 20)).astype(int)
    df['dia_fim_de_semana'] = (df['data_hora'].dt.dayofweek >= 5).astype(int)
    df['arredondamento_valor'] = (df['valor'] > 1000) & (df['valor'] % 1000 == 0).astype(int)
    df = df.sort_values(by=['id_entidade', 'data_hora'])
    diferenca_tempo = df.groupby('id_entidade')['data_hora'].diff().dt.total_seconds()
    df['sequencia_rapida'] = (diferenca_tempo < 60).astype(int)
    df.fillna(0, inplace=True)

    # --- Definição da Complexidade ---
    print("--- Definindo a Complexidade... ---")
    media_geral = df['valor'].mean()
    std_geral = df['valor'].std()
    criterio_1 = df['valor'] > (media_geral + 3 * std_geral)
    # Assumimos que a coluna 'conta' existe após o carregamento e renomeação
    criterio_2 = df['conta'].isin(['Diversos', 'Outros'])
    criterio_3 = df['valor'] < 0
    df['complexidade'] = np.where((criterio_1 | criterio_2 | criterio_3), 'Complexa', 'Simples')

    # --- Pré-processamento e Divisão ---
    features_para_modelo = df.select_dtypes(include=np.number).columns.drop(['id_transacao', 'id_entidade'], errors='ignore')
    X = df[features_para_modelo]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    df_scaled = pd.DataFrame(X_scaled, columns=features_para_modelo, index=df.index)
    df_scaled[['complexidade', 'id_transacao']] = df[['complexidade', 'id_transacao']]

    # --- Treino e Predição ---
    print("--- Treinando e prevendo com os modelos... ---")
    df['anomalia_detectada'] = 'Normal'
    df['modelo_deteccao'] = 'N/A'

    # Isolation Forest
    indices_simples = df_scaled[df_scaled['complexidade'] == 'Simples'].index
    df_simples = df_scaled.loc[indices_simples].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_simples.empty:
        iso_forest = IsolationForest(n_estimators=200, contamination=0.01, random_state=42, n_jobs=-1)
        iso_forest.fit(df_simples)
        predicoes_if = iso_forest.predict(df_simples)
        df.loc[indices_simples, 'anomalia_detectada'] = np.where(predicoes_if == -1, 'Anómala', 'Normal')
        df.loc[indices_simples, 'modelo_deteccao'] = 'Isolation Forest'

    # Autoencoder
    indices_complexas = df_scaled[df_scaled['complexidade'] == 'Complexa'].index
    df_complexas = df_scaled.loc[indices_complexas].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_complexas.empty:
        input_dim = df_complexas.shape[1]
        input_layer = Input(shape=(input_dim,)) # Este Input é do Keras
        encoder = Dense(16, activation='relu')(input_layer)
        encoder = Dense(8, activation='relu')(encoder)
        decoder = Dense(16, activation='relu')(encoder)
        decoder = Dense(input_dim, activation='linear')(decoder)
        autoencoder = Model(inputs=input_layer, outputs=decoder)
        autoencoder.compile(optimizer='adam', loss='mean_squared_error')
        autoencoder.fit(df_complexas, df_complexas, epochs=50, batch_size=32, verbose=0)
        reconstrucoes = autoencoder.predict(df_complexas)
        mse = np.mean(np.power(df_complexas.values - reconstrucoes, 2), axis=1)
        threshold = np.quantile(mse, 0.95)
        predicoes_ae = np.where(mse > threshold, -1, 1)
        df.loc[indices_complexas, 'anomalia_detectada'] = np.where(predicoes_ae == -1, 'Anómala', 'Normal')
        df.loc[indices_complexas, 'modelo_deteccao'] = 'Autoencoder'

    print("Pipeline de análise concluído.")
    return df.sort_index()

# ==============================================================================
# 3. INICIALIZAÇÃO E LAYOUT DA APLICAÇÃO DASH
# ==============================================================================
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Dashboard de Auditoria Financeira"

app.layout = dbc.Container([
    dcc.Store(id='dados-processados-memoria'),

    html.H1("Dashboard de Auditoria de Anomalias Financeiras", className="my-4 text-center"),

    dbc.Row([
        dbc.Col([
            dcc.Upload(
                id='upload-dados',
                children=html.Div(['Arraste e Solte ou ', html.A('Selecione um Ficheiro')]),
                style={
                    'width': '100%', 'height': '60px', 'lineHeight': '60px',
                    'borderWidth': '1px', 'borderStyle': 'dashed',
                    'borderRadius': '5px', 'textAlign': 'center', 'margin': '10px'
                },
                multiple=False
            ),
            html.Div(id='output-upload-state')
        ])
    ], className="mb-4"),

    html.Hr(),

    dbc.Row([
        dbc.Col([html.Label("Conta:"), dcc.Dropdown(id="conta-dropdown", value="Todos")], md=4),
        dbc.Col([html.Label("Tipo de Complexidade:"), dcc.Dropdown(id="tipo-dropdown", value="Todos")], md=4),
        dbc.Col([html.Label("Estado da Anomalia:"), dcc.Dropdown(id="anomalia-dropdown", value="Todas")], md=4)
    ], className="mb-4"),

    dcc.Graph(id="scatter-graph"),
    dash_table.DataTable(id="table", page_size=10, style_table={"overflowX": "auto"},
                         style_cell={"textAlign": "left"}, style_header={'backgroundColor': '#003366', 'color': 'white', 'fontWeight': 'bold'},
                         sort_action="native", filter_action="native")
], fluid=True)

# ==============================================================================
# 4. CALLBACKS - LÓGICA INTERATIVA DO DASHBOARD
# ==============================================================================

# Callback para processar o ficheiro carregado
@app.callback(
    Output('dados-processados-memoria', 'data'),
    Output('output-upload-state', 'children'),
    DashInput('upload-dados', 'contents'), # CORREÇÃO: Usar DashInput
    State('upload-dados', 'filename')
)
def processar_ficheiro_carregado(contents, filename):
    if contents is None:
        return None, "Por favor, carregue um ficheiro (CSV ou Excel) para iniciar a análise."

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)

    try:
        if 'csv' in filename:
            df_inicial = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename or 'xlsx' in filename:
            # Se o ficheiro Excel tiver lixo nas primeiras linhas, pode usar 'skiprows'
            # Ex: df_inicial = pd.read_excel(io.BytesIO(decoded), skiprows=3) para ignorar as 3 primeiras linhas
            df_inicial = pd.read_excel(io.BytesIO(decoded))
        else:
            return None, html.Div(['Tipo de ficheiro não suportado. Por favor, use CSV ou Excel.'], style={'color': 'red'})

        # --- INÍCIO DO BLOCO DE LIMPEZA AUTOMÁTICA ---

        # 1. Renomear colunas (exemplo: se o seu ficheiro tiver "Montante" em vez de "valor")
        colunas_para_renomear = {
            'ID da Transação': 'id_transacao',
            'Data': 'data_hora',
            'Montante': 'valor',
            'Categoria da Conta': 'conta',
            'Entidade': 'id_entidade',
            'Tipo': 'tipo_transacao'
        }
        # Renomeia apenas as colunas que existem no dicionário
        df_inicial.rename(columns=colunas_para_renomear, inplace=True)

        # 2. Converter colunas para os tipos corretos
        # Converte 'data_hora' para datetime, tratando possíveis erros
        df_inicial['data_hora'] = pd.to_datetime(df_inicial['data_hora'], errors='coerce')

        # Converte 'valor' para número, transformando o que não for número em Nulo (NaN)
        df_inicial['valor'] = pd.to_numeric(df_inicial['valor'], errors='coerce')

        # 3. Tratar valores em falta (exemplo: apagar linhas onde 'valor' ou 'data_hora' são nulos)
        df_inicial.dropna(subset=['valor', 'data_hora'], inplace=True)

        # --- FIM DO BLOCO DE LIMPEZA ---

        # Agora o df_inicial está pronto para o pipeline
        df_processado = executar_pipeline_de_analise(df_inicial)

        # Converter para JSON para armazenar no dcc.Store
        json_data = df_processado.to_json(date_format='iso', orient='split')

        return json_data, html.Div([f'Ficheiro "{filename}" carregado e processado com sucesso!'], style={'color': 'green'})

    except Exception as e:
        print(e)
        return None, html.Div(['Ocorreu um erro ao processar o ficheiro. Verifique se as colunas estão corretas (ex: "conta", "valor", "data_hora").'], style={'color': 'red'})

# Callback principal que atualiza o dashboard com base nos dados processados
@app.callback(
    Output("scatter-graph", "figure"),
    Output("table", "data"),
    Output("table", "columns"),
    Output("conta-dropdown", "options"),
    Output("tipo-dropdown", "options"),
    Output("anomalia-dropdown", "options"),
    DashInput('dados-processados-memoria', 'data'), # CORREÇÃO: Usar DashInput
    DashInput("conta-dropdown", "value"),
    DashInput("tipo-dropdown", "value"),
    DashInput("anomalia-dropdown", "value")
)
def actualizar_dashboard(json_data, conta, tipo, anomalia):
    # Estado inicial antes de carregar um ficheiro ou se ocorrer um erro
    if json_data is None:
        fig_vazia = {"layout": {"xaxis": {"visible": False}, "yaxis": {"visible": False}, "annotations": [{"text": "Nenhum dado para exibir. Por favor, carregue um ficheiro.", "xref": "paper", "yref": "paper", "showarrow": False, "font": {"size": 20}}]}}
        colunas_vazias = [{"name": col, "id": col} for col in ["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]]
        opcoes_vazias = [{"label": "Todos", "value": "Todos"}]
        opcoes_anomalia_fixas = [
            {"label": "Todas", "value": "Todas"},
            {"label": "Apenas Anómalas", "value": "Anómalas"},
            {"label": "Apenas Normais", "value": "Normais"}
        ]
        return fig_vazia, [], colunas_vazias, opcoes_vazias, opcoes_vazias, opcoes_anomalia_fixas

    df_final = pd.read_json(json_data, orient='split')
    df_final['data_hora'] = pd.to_datetime(df_final['data_hora'])

    # Atualizar opções dos filtros
    opcoes_conta = [{"label": c, "value": c} for c in ["Todos"] + sorted(df_final["conta"].unique())]
    opcoes_tipo = [{"label": c, "value": c} for c in ["Todos"] + df_final["complexidade"].unique().tolist()]
    opcoes_anomalia = [
        {"label": "Todas", "value": "Todas"},
        {"label": "Apenas Anómalas", "value": "Anómalas"},
        {"label": "Apenas Normais", "value": "Normais"}
    ]

    # Lógica de filtragem
    df_filtrado = df_final.copy()
    if conta and conta != "Todos":
        df_filtrado = df_filtrado[df_filtrado["conta"] == conta]
    if tipo and tipo != "Todos":
        df_filtrado = df_filtrado[df_filtrado["complexidade"] == tipo]
    if anomalia and anomalia == "Anómalas":
        df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Anómala"]
    elif anomalia and anomalia == "Normais":
        df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Normal"]

    # Criar o gráfico
    fig = px.scatter(
        df_filtrado, x="valor", y="dia_do_ano",
        color="anomalia_detectada",
        hover_data=['conta', 'complexidade', 'modelo_deteccao', 'data_hora'],
        color_discrete_map={'Anómala': '#FF4136', 'Normal': '#0074D9'},
        title="Visualização de Anomalias (Valor vs. Dia do Ano)",
        labels={"valor": "Valor da Transação (MZN)", "dia_do_ano": "Dia do Ano"}
    )
    fig.update_layout(transition_duration=500)

    # Preparar dados para a tabela
    colunas_tabela = [{"name": col, "id": col} for col in ["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]]
    data_tabela = df_filtrado[ [c['id'] for c in colunas_tabela] ].to_dict("records")

    return fig, data_tabela, colunas_tabela, opcoes_conta, opcoes_tipo, opcoes_anomalia

# ==============================================================================
# 5. EXECUÇÃO DA APLICAÇÃO NO COLAB
# ==============================================================================
if __name__ == "__main__":
    app.run_server(mode='inline')

Bibliotecas importadas com sucesso.
Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



<IPython.core.display.Javascript object>

# **----------------------------ALGORITMO FUNCIONAL---------------------**

In [ ]:
# ==============================================================================
# 0. SETUP - INSTALAÇÃO DE VERSÕES ESPECÍFICAS E COMPATÍVEIS
# ==============================================================================
# Instalamos versões específicas que são conhecidas por funcionarem bem juntas.
!pip install "dash==2.10.2" "jupyter-dash==0.4.2" dash-bootstrap-components plotly openpyxl -q

# ==============================================================================
# 1. SETUP - IMPORTAÇÃO DAS BIBLIOTECAS
# ==============================================================================
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
from dash.dependencies import Input as DashInput, Output, State
import plotly.express as px
import pandas as pd
import numpy as np
import base64
import io

# Bibliotecas de Machine Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

print("Bibliotecas importadas com sucesso.")

# ==============================================================================
# 2. FUNÇÃO DO PIPELINE DE DADOS E MODELAGEM
# ==============================================================================
def executar_pipeline_de_analise(df_inicial):
    """
    Executa o pipeline completo sobre um DataFrame fornecido.
    Retorna um DataFrame final com os resultados da análise.
    """
    print("\n--- A executar o pipeline de análise sobre os novos dados... ---")
    df = df_inicial.copy()

    # --- Engenharia de Atributos ---
    print("--- Engenharia de Atributos... ---")
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    df['dia_do_ano'] = df['data_hora'].dt.dayofyear
    mediana_por_entidade = df.groupby('id_entidade')['valor'].transform('median')
    df['desvio_mediana_historica'] = (df['valor'] - mediana_por_entidade) / (mediana_por_entidade + 1e-6)
    df['hora_incomum'] = ((df['data_hora'].dt.hour < 7) | (df['data_hora'].dt.hour > 20)).astype(int)
    df['dia_fim_de_semana'] = (df['data_hora'].dt.dayofweek >= 5).astype(int)
    df['arredondamento_valor'] = (df['valor'] > 1000) & (df['valor'] % 1000 == 0).astype(int)
    df = df.sort_values(by=['id_entidade', 'data_hora'])
    diferenca_tempo = df.groupby('id_entidade')['data_hora'].diff().dt.total_seconds()
    df['sequencia_rapida'] = (diferenca_tempo < 60).astype(int)
    df.fillna(0, inplace=True)

    # --- Definição da Complexidade ---
    print("--- Definindo a Complexidade... ---")
    media_geral = df['valor'].mean()
    std_geral = df['valor'].std()
    criterio_1 = df['valor'] > (media_geral + 3 * std_geral)
    criterio_2 = df['conta'].isin(['Diversos', 'Outros'])
    criterio_3 = df['valor'] < 0
    df['complexidade'] = np.where((criterio_1 | criterio_2 | criterio_3), 'Complexa', 'Simples')

    # --- Pré-processamento e Divisão ---
    features_para_modelo = df.select_dtypes(include=np.number).columns.drop(['id_transacao', 'id_entidade'], errors='ignore')
    X = df[features_para_modelo]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    df_scaled = pd.DataFrame(X_scaled, columns=features_para_modelo, index=df.index)
    df_scaled[['complexidade', 'id_transacao']] = df[['complexidade', 'id_transacao']]

    # --- Treino e Predição ---
    print("--- Treinando e prevendo com os modelos... ---")
    df['anomalia_detectada'] = 'Normal'
    df['modelo_deteccao'] = 'N/A'

    # Isolation Forest
    indices_simples = df_scaled[df_scaled['complexidade'] == 'Simples'].index
    df_simples = df_scaled.loc[indices_simples].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_simples.empty:
        iso_forest = IsolationForest(n_estimators=200, contamination=0.01, random_state=42, n_jobs=-1)
        iso_forest.fit(df_simples)
        predicoes_if = iso_forest.predict(df_simples)
        df.loc[indices_simples, 'anomalia_detectada'] = np.where(predicoes_if == -1, 'Anómala', 'Normal')
        df.loc[indices_simples, 'modelo_deteccao'] = 'Isolation Forest'

    # Autoencoder
    indices_complexas = df_scaled[df_scaled['complexidade'] == 'Complexa'].index
    df_complexas = df_scaled.loc[indices_complexas].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_complexas.empty:
        input_dim = df_complexas.shape[1]
        input_layer = Input(shape=(input_dim,))
        encoder = Dense(16, activation='relu')(input_layer)
        encoder = Dense(8, activation='relu')(encoder)
        decoder = Dense(16, activation='relu')(encoder)
        decoder = Dense(input_dim, activation='linear')(decoder)
        autoencoder = Model(inputs=input_layer, outputs=decoder)
        autoencoder.compile(optimizer='adam', loss='mean_squared_error')
        autoencoder.fit(df_complexas, df_complexas, epochs=50, batch_size=32, verbose=0)
        reconstrucoes = autoencoder.predict(df_complexas)
        mse = np.mean(np.power(df_complexas.values - reconstrucoes, 2), axis=1)
        threshold = np.quantile(mse, 0.95)
        predicoes_ae = np.where(mse > threshold, -1, 1)
        df.loc[indices_complexas, 'anomalia_detectada'] = np.where(predicoes_ae == -1, 'Anómala', 'Normal')
        df.loc[indices_complexas, 'modelo_deteccao'] = 'Autoencoder'

    print("Pipeline de análise concluído.")
    return df.sort_index()

# ==============================================================================
# 3. INICIALIZAÇÃO E LAYOUT DA APLICAÇÃO DASH
# ==============================================================================
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Dashboard de Auditoria Financeira"

app.layout = dbc.Container([
    dcc.Store(id='dados-processados-memoria'),

    html.H1("Dashboard de Auditoria de Anomalias Financeiras", className="my-4 text-center"),

    dbc.Row([
        dbc.Col([
            dcc.Upload(
                id='upload-dados',
                children=html.Div(['Arraste e Solte ou ', html.A('Selecione um Ficheiro')]),
                style={
                    'width': '100%', 'height': '60px', 'lineHeight': '60px',
                    'borderWidth': '1px', 'borderStyle': 'dashed',
                    'borderRadius': '5px', 'textAlign': 'center', 'margin': '10px'
                },
                multiple=False
            ),
            html.Div(id='output-upload-state')
        ])
    ], className="mb-4"),

    html.Hr(),

    dbc.Row([
        dbc.Col([html.Label("Conta:"), dcc.Dropdown(id="conta-dropdown", value="Todos")], md=4),
        dbc.Col([html.Label("Tipo de Complexidade:"), dcc.Dropdown(id="tipo-dropdown", value="Todos")], md=4),
        dbc.Col([html.Label("Estado da Anomalia:"), dcc.Dropdown(id="anomalia-dropdown", value="Todas")], md=4)
    ], className="mb-4"),

    dcc.Graph(id="scatter-graph"),
    dash_table.DataTable(id="table", page_size=10, style_table={"overflowX": "auto"},
                         style_cell={"textAlign": "left"}, style_header={'backgroundColor': '#003366', 'color': 'white', 'fontWeight': 'bold'},
                         sort_action="native", filter_action="native")
], fluid=True)

# ==============================================================================
# 4. CALLBACKS - LÓGICA INTERATIVA DO DASHBOARD
# ==============================================================================

# Callback para processar o ficheiro carregado
@app.callback(
    Output('dados-processados-memoria', 'data'),
    Output('output-upload-state', 'children'),
    DashInput('upload-dados', 'contents'),
    State('upload-dados', 'filename')
)
def processar_ficheiro_carregado(contents, filename):
    if contents is None:
        return None, "Por favor, carregue um ficheiro (CSV ou Excel) para iniciar a análise."

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)

    try:
        if 'csv' in filename:
            df_inicial = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename or 'xlsx' in filename:
            df_inicial = pd.read_excel(io.BytesIO(decoded))
        else:
            return None, html.Div(['Tipo de ficheiro não suportado. Por favor, use CSV ou Excel.'], style={'color': 'red'})

        # --- INÍCIO DO BLOCO DE LIMPEZA AUTOMÁTICA ---

        # !! IMPORTANTE: AJUSTE ESTE DICIONÁRIO PARA OS NOMES DAS SUAS COLUNAS !!
        colunas_para_renomear = {
            'ID da Transação': 'id_transacao',
            'Data': 'data_hora',
            'Montante': 'valor',
            'Categoria da Conta': 'conta',
            'Entidade': 'id_entidade',
            'Tipo': 'tipo_transacao'
        }
        df_inicial.rename(columns=colunas_para_renomear, inplace=True)

        # Validação das colunas essenciais
        colunas_necessarias = ['id_transacao', 'data_hora', 'valor', 'conta', 'id_entidade']
        colunas_em_falta = [col for col in colunas_necessarias if col not in df_inicial.columns]
        if colunas_em_falta:
            return None, html.Div([f'Erro: Faltam as seguintes colunas no ficheiro: {", ".join(colunas_em_falta)}'], style={'color': 'red'})

        df_inicial['data_hora'] = pd.to_datetime(df_inicial['data_hora'], errors='coerce')
        df_inicial['valor'] = pd.to_numeric(df_inicial['valor'], errors='coerce')
        df_inicial.dropna(subset=['valor', 'data_hora'], inplace=True)

        # --- FIM DO BLOCO DE LIMPEZA ---

        df_processado = executar_pipeline_de_analise(df_inicial)
        json_data = df_processado.to_json(date_format='iso', orient='split')

        return json_data, html.Div([f'Ficheiro "{filename}" carregado e processado com sucesso!'], style={'color': 'green'})

    except Exception as e:
        print(e)
        return None, html.Div(['Ocorreu um erro ao processar o ficheiro. Verifique o formato e as colunas.'], style={'color': 'red'})

# Callback principal que atualiza o dashboard
@app.callback(
    Output("scatter-graph", "figure"),
    Output("table", "data"),
    Output("table", "columns"),
    Output("conta-dropdown", "options"),
    Output("tipo-dropdown", "options"),
    Output("anomalia-dropdown", "options"),
    DashInput('dados-processados-memoria', 'data'),
    DashInput("conta-dropdown", "value"),
    DashInput("tipo-dropdown", "value"),
    DashInput("anomalia-dropdown", "value")
)
def actualizar_dashboard(json_data, conta, tipo, anomalia):
    if json_data is None:
        fig_vazia = {"layout": {"xaxis": {"visible": False}, "yaxis": {"visible": False}, "annotations": [{"text": "Nenhum dado para exibir. Carregue um ficheiro.", "xref": "paper", "yref": "paper", "showarrow": False, "font": {"size": 20}}]}}
        colunas_vazias = [{"name": col, "id": col} for col in ["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]]
        opcoes_vazias = [{"label": "Todos", "value": "Todos"}]
        opcoes_anomalia_fixas = [{"label": "Todas", "value": "Todas"}, {"label": "Apenas Anômalas", "value": "Anómalas"}, {"label": "Apenas Normais", "value": "Normais"}]
        return fig_vazia, [], colunas_vazias, opcoes_vazias, opcoes_vazias, opcoes_anomalia_fixas

    df_final = pd.read_json(json_data, orient='split')
    df_final['data_hora'] = pd.to_datetime(df_final['data_hora'])

    opcoes_conta = [{"label": c, "value": c} for c in ["Todos"] + sorted(df_final["conta"].unique())]
    opcoes_tipo = [{"label": c, "value": c} for c in ["Todos"] + df_final["complexidade"].unique().tolist()]
    opcoes_anomalia = [{"label": "Todas", "value": "Todas"}, {"label": "Apenas Anômalas", "value": "Anómalas"}, {"label": "Apenas Normais", "value": "Normais"}]

    df_filtrado = df_final.copy()
    if conta and conta != "Todos":
        df_filtrado = df_filtrado[df_filtrado["conta"] == conta]
    if tipo and tipo != "Todos":
        df_filtrado = df_filtrado[df_filtrado["complexidade"] == tipo]
    if anomalia and anomalia == "Anómalas":
        df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Anómala"]
    elif anomalia and anomalia == "Normais":
        df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Normal"]

    fig = px.scatter(
        df_filtrado, x="valor", y="dia_do_ano",
        color="anomalia_detectada",
        hover_data=['conta', 'complexidade', 'modelo_deteccao', 'data_hora'],
        color_discrete_map={'Anómala': '#FF4136', 'Normal': '#0074D9'},
        title="Visualização de Anomalias (Valor vs. Dia do Ano)",
        labels={"valor": "Valor da Transação (MZN)", "dia_do_ano": "Dia do Ano"}
    )
    fig.update_layout(transition_duration=500)

    colunas_tabela = [{"name": col, "id": col} for col in ["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]]
    data_tabela = df_filtrado[[c['id'] for c in colunas_tabela]].to_dict("records")

    return fig, data_tabela, colunas_tabela, opcoes_conta, opcoes_tipo, opcoes_anomalia

# ==============================================================================
# 5. EXECUÇÃO DA APLICAÇÃO NO COLAB
# ==============================================================================
if __name__ == "__main__":
    app.run_server(mode='inline')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.1 MB/s eta 0:00:00
Bibliotecas importadas com sucesso.
Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



<IPython.core.display.Javascript object>

# **---------------ALGORITMO FUNCIONAL---------** ⏫::::

# **-----------------------ALGORITMO FUNCIONAL C/ FUNCIONALIDADE DE BAIXAR RELATORIO------------------------------------------------------------**:  ⬇ ⏬

In [ ]:
# ==============================================================================
# 0. SETUP - INSTALAÇÃO DE VERSÕES ESPECÍFICAS E COMPATÍVEIS
# ==============================================================================
# Instalamos versões específicas que são conhecidas por funcionarem bem juntas.
!pip install "dash==2.10.2" "jupyter-dash==0.4.2" dash-bootstrap-components plotly openpyxl -q

# ==============================================================================
# 1. SETUP - IMPORTAÇÃO DAS BIBLIOTECAS
# ==============================================================================
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc
from dash import dcc, html, dash_table
from dash.dependencies import Input as DashInput, Output, State
import plotly.express as px
import pandas as pd
import numpy as np
import base64
import io

# Bibliotecas de Machine Learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import IsolationForest
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

print("Bibliotecas importadas com sucesso.")

# ==============================================================================
# 2. FUNÇÃO DO PIPELINE DE DADOS E MODELAGEM
# ==============================================================================
def executar_pipeline_de_analise(df_inicial):
    """
    Executa o pipeline completo sobre um DataFrame fornecido.
    Retorna um DataFrame final com os resultados da análise.
    """
    print("\n--- A executar o pipeline de análise sobre os novos dados... ---")
    df = df_inicial.copy()

    # --- Engenharia de Atributos ---
    print("--- Engenharia de Atributos... ---")
    df['data_hora'] = pd.to_datetime(df['data_hora'])
    df['dia_do_ano'] = df['data_hora'].dt.dayofyear
    mediana_por_entidade = df.groupby('id_entidade')['valor'].transform('median')
    df['desvio_mediana_historica'] = (df['valor'] - mediana_por_entidade) / (mediana_por_entidade + 1e-6)
    df['hora_incomum'] = ((df['data_hora'].dt.hour < 7) | (df['data_hora'].dt.hour > 20)).astype(int)
    df['dia_fim_de_semana'] = (df['data_hora'].dt.dayofweek >= 5).astype(int)
    df['arredondamento_valor'] = (df['valor'] > 1000) & (df['valor'] % 1000 == 0).astype(int)
    df = df.sort_values(by=['id_entidade', 'data_hora'])
    diferenca_tempo = df.groupby('id_entidade')['data_hora'].diff().dt.total_seconds()
    df['sequencia_rapida'] = (diferenca_tempo < 60).astype(int)
    df.fillna(0, inplace=True)

    # --- Definição da Complexidade ---
    print("--- Definindo a Complexidade... ---")
    media_geral = df['valor'].mean()
    std_geral = df['valor'].std()
    criterio_1 = df['valor'] > (media_geral + 3 * std_geral)
    criterio_2 = df['conta'].isin(['Diversos', 'Outros'])
    criterio_3 = df['valor'] < 0
    df['complexidade'] = np.where((criterio_1 | criterio_2 | criterio_3), 'Complexa', 'Simples')

    # --- Pré-processamento e Divisão ---
    features_para_modelo = df.select_dtypes(include=np.number).columns.drop(['id_transacao', 'id_entidade'], errors='ignore')
    X = df[features_para_modelo]
    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    df_scaled = pd.DataFrame(X_scaled, columns=features_para_modelo, index=df.index)
    df_scaled[['complexidade', 'id_transacao']] = df[['complexidade', 'id_transacao']]

    # --- Treino e Predição ---
    print("--- Treinando e prevendo com os modelos... ---")
    df['anomalia_detectada'] = 'Normal'
    df['modelo_deteccao'] = 'N/A'

    # Isolation Forest
    indices_simples = df_scaled[df_scaled['complexidade'] == 'Simples'].index
    df_simples = df_scaled.loc[indices_simples].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_simples.empty:
        iso_forest = IsolationForest(n_estimators=200, contamination=0.01, random_state=42, n_jobs=-1)
        iso_forest.fit(df_simples)
        predicoes_if = iso_forest.predict(df_simples)
        df.loc[indices_simples, 'anomalia_detectada'] = np.where(predicoes_if == -1, 'Anómala', 'Normal')
        df.loc[indices_simples, 'modelo_deteccao'] = 'Isolation Forest'

    # Autoencoder
    indices_complexas = df_scaled[df_scaled['complexidade'] == 'Complexa'].index
    df_complexas = df_scaled.loc[indices_complexas].drop(['complexidade', 'id_transacao'], axis=1)
    if not df_complexas.empty:
        input_dim = df_complexas.shape[1]
        input_layer = Input(shape=(input_dim,))
        encoder = Dense(16, activation='relu')(input_layer)
        encoder = Dense(8, activation='relu')(encoder)
        decoder = Dense(16, activation='relu')(encoder)
        decoder = Dense(input_dim, activation='linear')(decoder)
        autoencoder = Model(inputs=input_layer, outputs=decoder)
        autoencoder.compile(optimizer='adam', loss='mean_squared_error')
        autoencoder.fit(df_complexas, df_complexas, epochs=50, batch_size=32, verbose=0)
        reconstrucoes = autoencoder.predict(df_complexas)
        mse = np.mean(np.power(df_complexas.values - reconstrucoes, 2), axis=1)
        threshold = np.quantile(mse, 0.95)
        predicoes_ae = np.where(mse > threshold, -1, 1)
        df.loc[indices_complexas, 'anomalia_detectada'] = np.where(predicoes_ae == -1, 'Anómala', 'Normal')
        df.loc[indices_complexas, 'modelo_deteccao'] = 'Autoencoder'

    print("Pipeline de análise concluído.")
    return df.sort_index()

# ==============================================================================
# 3. INICIALIZAÇÃO E LAYOUT DA APLICAÇÃO DASH
# ==============================================================================
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = "Dashboard de Auditoria Financeira"

app.layout = dbc.Container([
    dcc.Store(id='dados-processados-memoria'),
    dcc.Download(id="download-relatorio-csv"),

    html.H1("Dashboard de Auditoria de Anomalias Financeiras", className="my-4 text-center"),

    dbc.Row([
        dbc.Col([
            dcc.Upload(
                id='upload-dados',
                children=html.Div(['Arraste e Solte ou ', html.A('Selecione um Ficheiro')]),
                style={
                    'width': '100%', 'height': '60px', 'lineHeight': '60px',
                    'borderWidth': '1px', 'borderStyle': 'dashed',
                    'borderRadius': '5px', 'textAlign': 'center', 'margin': '10px'
                },
                multiple=False
            ),
            html.Div(id='output-upload-state')
        ])
    ], className="mb-4"),

    html.Hr(),

    dbc.Row([
        dbc.Col([html.Label("Conta:"), dcc.Dropdown(id="conta-dropdown", value="Todos")], md=3),
        dbc.Col([html.Label("Tipo de Complexidade:"), dcc.Dropdown(id="tipo-dropdown", value="Todos")], md=3),
        dbc.Col([html.Label("Estado da Anomalia:"), dcc.Dropdown(id="anomalia-dropdown", value="Todas")], md=3),
        dbc.Col([
            html.Label("Ação:"),
            html.Button("Baixar Relatório (CSV)", id="btn-baixar-csv", className="w-100 btn btn-success")
        ], md=3, className="d-flex flex-column justify-content-end")
    ], className="mb-4"),

    dcc.Graph(id="scatter-graph"),
    dash_table.DataTable(id="table", page_size=10, style_table={"overflowX": "auto"},
                         style_cell={"textAlign": "left"}, style_header={'backgroundColor': '#003366', 'color': 'white', 'fontWeight': 'bold'},
                         sort_action="native", filter_action="native")
], fluid=True)

# ==============================================================================
# 4. CALLBACKS - LÓGICA INTERATIVA DO DASHBOARD
# ==============================================================================

# Callback para processar o ficheiro carregado
@app.callback(
    Output('dados-processados-memoria', 'data'),
    Output('output-upload-state', 'children'),
    DashInput('upload-dados', 'contents'),
    State('upload-dados', 'filename')
)
def processar_ficheiro_carregado(contents, filename):
    if contents is None:
        return None, "Por favor, carregue um ficheiro (CSV ou Excel) para iniciar a análise."

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)

    try:
        if 'csv' in filename:
            df_inicial = pd.read_csv(io.StringIO(decoded.decode('utf-8')))
        elif 'xls' in filename or 'xlsx' in filename:
            df_inicial = pd.read_excel(io.BytesIO(decoded))
        else:
            return None, html.Div(['Tipo de ficheiro não suportado.'], style={'color': 'red'})

        # --- INÍCIO DO BLOCO DE LIMPEZA AUTOMÁTICA ---

        # !! IMPORTANTE: AJUSTE ESTE DICIONÁRIO PARA OS NOMES DAS SUAS COLUNAS !!
        colunas_para_renomear = {
            'ID da Transação': 'id_transacao',
            'Data': 'data_hora',
            'Montante': 'valor',
            'Categoria da Conta': 'conta',
            'Entidade': 'id_entidade',
            'Tipo': 'tipo_transacao'
        }
        df_inicial.rename(columns=colunas_para_renomear, inplace=True)

        colunas_necessarias = ['id_transacao', 'data_hora', 'valor', 'conta', 'id_entidade']
        colunas_em_falta = [col for col in colunas_necessarias if col not in df_inicial.columns]
        if colunas_em_falta:
            return None, html.Div([f'Erro: Faltam as seguintes colunas: {", ".join(colunas_em_falta)}'], style={'color': 'red'})

        df_inicial['data_hora'] = pd.to_datetime(df_inicial['data_hora'], errors='coerce')
        df_inicial['valor'] = pd.to_numeric(df_inicial['valor'], errors='coerce')
        df_inicial.dropna(subset=['valor', 'data_hora'], inplace=True)

        # --- FIM DO BLOCO DE LIMPEZA ---

        df_processado = executar_pipeline_de_analise(df_inicial)
        json_data = df_processado.to_json(date_format='iso', orient='split')

        return json_data, html.Div([f'Ficheiro "{filename}" carregado e processado com sucesso!'], style={'color': 'green'})

    except Exception as e:
        print(e)
        return None, html.Div(['Ocorreu um erro ao processar o ficheiro. Verifique o formato e as colunas.'], style={'color': 'red'})

# Callback principal que atualiza o dashboard
@app.callback(
    Output("scatter-graph", "figure"),
    Output("table", "data"),
    Output("table", "columns"),
    Output("conta-dropdown", "options"),
    Output("tipo-dropdown", "options"),
    Output("anomalia-dropdown", "options"),
    DashInput('dados-processados-memoria', 'data'),
    DashInput("conta-dropdown", "value"),
    DashInput("tipo-dropdown", "value"),
    DashInput("anomalia-dropdown", "value")
)
def actualizar_dashboard(json_data, conta, tipo, anomalia):
    if json_data is None:
        fig_vazia = {"layout": {"annotations": [{"text": "Carregue um ficheiro para exibir os dados.", "xref": "paper", "yref": "paper", "showarrow": False, "font": {"size": 20}}]}}
        colunas_vazias = [{"name": col, "id": col} for col in ["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]]
        opcoes_vazias = [{"label": "Todos", "value": "Todos"}]
        opcoes_anomalia_fixas = [{"label": "Todas", "value": "Todas"},{"label": "Apenas Anômalas", "value": "Anómalas"},{"label": "Apenas Normais", "value": "Normais"}]
        return fig_vazia, [], colunas_vazias, opcoes_vazias, opcoes_vazias, opcoes_anomalia_fixas

    df_final = pd.read_json(json_data, orient='split')
    df_final['data_hora'] = pd.to_datetime(df_final['data_hora'])

    opcoes_conta = [{"label": c, "value": c} for c in ["Todos"] + sorted(df_final["conta"].unique())]
    opcoes_tipo = [{"label": c, "value": c} for c in ["Todos"] + df_final["complexidade"].unique().tolist()]
    opcoes_anomalia = [{"label": "Todas", "value": "Todas"},{"label": "Apenas Anômalas", "value": "Anómalas"},{"label": "Apenas Normais", "value": "Normais"}]

    df_filtrado = df_final.copy()
    if conta and conta != "Todos": df_filtrado = df_filtrado[df_filtrado["conta"] == conta]
    if tipo and tipo != "Todos": df_filtrado = df_filtrado[df_filtrado["complexidade"] == tipo]
    if anomalia and anomalia == "Anómalas": df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Anómala"]
    elif anomalia and anomalia == "Normais": df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Normal"]

    fig = px.scatter(
        df_filtrado, x="valor", y="dia_do_ano", color="anomalia_detectada",
        hover_data=['conta', 'complexidade', 'modelo_deteccao', 'data_hora'],
        color_discrete_map={'Anómala': '#FF4136', 'Normal': '#0074D9'},
        title="Visualização de Anomalias (Valor vs. Dia do Ano)",
        labels={"valor": "Valor da Transação (MZN)", "dia_do_ano": "Dia do Ano"}
    )
    fig.update_layout(transition_duration=500)

    colunas_tabela = [{"name": col, "id": col} for col in ["conta", "valor", "data_hora", "complexidade", "anomalia_detectada", "modelo_deteccao"]]
    data_tabela = df_filtrado[[c['id'] for c in colunas_tabela]].to_dict("records")

    return fig, data_tabela, colunas_tabela, opcoes_conta, opcoes_tipo, opcoes_anomalia

# Callback para gerar e baixar o relatório CSV
@app.callback(
    Output("download-relatorio-csv", "data"),
    DashInput("btn-baixar-csv", "n_clicks"),
    State('dados-processados-memoria', 'data'),
    State("conta-dropdown", "value"),
    State("tipo-dropdown", "value"),
    State("anomalia-dropdown", "value"),
    prevent_initial_call=True
)
def baixar_relatorio(n_clicks, json_data, conta, tipo, anomalia):
    if json_data is None:
        return None

    df_final = pd.read_json(json_data, orient='split')

    df_filtrado = df_final.copy()
    if conta and conta != "Todos": df_filtrado = df_filtrado[df_filtrado["conta"] == conta]
    if tipo and tipo != "Todos": df_filtrado = df_filtrado[df_filtrado["complexidade"] == tipo]
    if anomalia and anomalia == "Anómalas": df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Anómala"]
    elif anomalia and anomalia == "Normais": df_filtrado = df_filtrado[df_filtrado["anomalia_detectada"] == "Normal"]

    return dcc.send_data_frame(df_filtrado.to_csv, "relatorio_auditoria.csv", index=False)


# ==============================================================================
# 5. EXECUÇÃO DA APLICAÇÃO NO COLAB
# ==============================================================================
if __name__ == "__main__":
    app.run_server(mode='inline')

Bibliotecas importadas com sucesso.
Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



<IPython.core.display.Javascript object>

# **⏫: ⏫  ALGORITMO FUNCIONAL V2**